Function signle_pad_analysis is too large to analyse

In [ ]:
import uproot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from parametrize import fff, ffm

In [ ]:
from matplotlib.patches import Rectangle
import scipy.stats as sts
from scipy.stats import norm
from sklearn.neighbors import KernelDensity
import seaborn as sns
from scipy.optimize import curve_fit, minimize, fsolve
from copy import deepcopy
import ctypes
from root_to_np import get_tree, get_df, rename_tree_branches

In [ ]:
!pip install SciencePlots

In [3]:
#plt.style.use(['notebook', 'science'])
plt.style.use(['default'])
plt.rcParams['legend.frameon'] = True
plt.rcParams['legend.title_fontsize'] = 20
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.labelsize'] = 20
plt.rcParams['text.usetex'] = False
plt.rcParams['xtick.labelsize'] = 20
plt.rcParams['ytick.labelsize'] = 20
plt.rcParams['legend.framealpha'] = 1

plt.rcParams['axes.titlesize'] = 22
plt.rcParams['axes.titleweight'] = 'bold'

# plt.rcParams.keys()

In [4]:
import ROOT

## Fit Functions

In [5]:
def expo1(x, pars):
    return ROOT.TMath.Exp(x[0]*pars[0]+pars[1])+pars[2]
    
def expo1_np(x, pars):
    return np.exp(x*pars[0]+pars[1])+pars[2]


def expo2(x, pars):
    return ROOT.TMath.Exp(pars[0]*x[0] + pars[1]) + ROOT.TMath.Exp(pars[2]*x[0] + pars[3]) + pars[4]

def expo2_np(x, pars):
    return np.exp(pars[0]*x + pars[1]) + np.exp(pars[2]*x + pars[3]) + pars[4]

def pol2(x, a, b, c):
    return a*(x-b)*(x-b)+c

# SAT vs parameters for twalk correction

In [6]:
cc_sat = ROOT.TCanvas('sat_fit','', 1000, 800)
def sat_fit(xpar, ypar, expar, eypar, iistart=0, iistop=None, xl=0.02, xu=0.5, pars=None):
#     iistart, iistop = 0, None
    cc_sat.cd(0)
    grcci = ROOT.TGraphErrors(len(xpar[iistart:iistop]), xpar[iistart:iistop], ypar[iistart:iistop], expar[iistart:iistop], eypar[iistart:iistop])
    grcci.SetNameTitle('grcci', ' ;Charge [pC];SAT [ps]')


    fffi = ROOT.TF1('ffsat', expo2, xl, xu, 5)
    wFiti = ROOT.Math.WrappedMultiTF1(fffi, 1);
    fopt =  ROOT.Fit.DataOptions()
    fopt.fCoordErrors = False
    rangeB = ROOT.Fit.DataRange()
    if iistop == None:
        rangeB.SetRange(xpar[iistart], xpar[-1])
    else:
        rangeB.SetRange(xpar[iistart], xpar[iistop])
    dataB = ROOT.Fit.BinData(fopt, rangeB);
    ROOT.Fit.FillData(dataB, grcci);
    chi2f = ROOT.Fit.Chi2Function(dataB, wFiti)

    fitter = ROOT.Fit.Fitter()

    par0 = np.array([-1.19593947e+01,  1.52272605e+01, -5.23861833e-01,  5.09975903e+00, 100])
    if pars != None:
        par0 = np.array(pars)
        
    fitter.Config().SetParamsSettings(5, par0)
    fitter.Config().MinimizerOptions().SetPrintLevel(1)
    fitter.Config().MinimizerOptions().SetMaxFunctionCalls(100000) # for Minuit/Minuit2
    fitter.Config().MinimizerOptions().SetMaxIterations(10000) # for GSL
    fitter.Config().SetMinimizer("Minuit2", "Migrad")

    fitter.FitFCN(chi2f)
    result = fitter.Result()

    parsi = np.zeros(5)
    for i in range(5):
        parsi[i] = fffi.GetParameter(i)
#     print(parsi)


    grcci.GetListOfFunctions().Add(fffi)
    grcci.DrawClone('ap')

    cc_sat.Draw()
    return parsi

## Change from Beam Frame to Local  Frame Functions

In [7]:
# def beam_coords(i, params):
#     d = 0.20
#     x0, y0, u = params
#     xl = np.sign(xdata[i] - (-x0)) * (d + 9.82) / 2
#     yl = np.sign(ydata[i] - (-y0)) * (d + 9.82) / 2
    
#     # Ensure that xl and yl are not both zero to avoid division by zero
#     if xl == 0 and yl == 0:
#         xl += 1e-10
    
#     return ((xl - x0) * np.cos(u) + (yl - y0) * np.sin(u), -(xl - x0) * np.sin(u) + (yl - y0) * np.cos(u))

def beam_coords(i,params):
    d =0.20
    x0,y0,u = params
    xl = np.sign(xdata[i]-(-x0))*(d+9.82)/2
    yl = np.sign(ydata[i]-(-y0))*(d+9.82)/2
    return((xl - x0)*np.cos(u)+(yl-y0)*np.sin(u), -(xl - x0)*np.sin(u)+(yl-y0)*np.cos(u))

#def chi2_i(i,params):
#    xb, yb = beam_coords(i,params) 
#    return ((xdata[i]-xb)/sigmax[i])**2+((ydata[i]-yb)/sigmay[i])**2
def chi2_i(i, params):
    xb, yb = beam_coords(i, params)
    
    # Check if sigmax or sigmay is zero, and handle it
    if sigmax[i] == 0 or sigmay[i] == 0:
        # Handle the case where sigmax or sigmay is zero, e.g., set the corresponding term to zero
        return 0.0
    else:
        return ((xdata[i] - xb) / sigmax[i])**2 + ((ydata[i] - yb) / sigmay[i])**2

def chi2(params):
    return chi2_i(0,params)+chi2_i(1,params)+chi2_i(2,params)+chi2_i(3,params)

def weird_gaussian( x, p ):
    #c, mu1, mu2, sigma = params
    res =   p[0] * (np.exp( - (x[0] - p[1])**2.0 / (2.0 * p[3]**2.0) ) \
          + np.exp( - (x[0] - p[2])**2.0 / (2.0 * p[3]**2.0) ))
    return res

def weirdo_gaussian( x, *p ):
    c, mu1, mu2, sigma = p
    res =   c * (np.exp( - (x - mu1)**2.0 / (2.0 * sigma**2.0) ) \
          + np.exp( - (x - mu2)**2.0 / (2.0 * sigma**2.0) ))
    return res

def parabola( x, p ):
    res =   p[0] * (x[0]-p[1])**2 +p[2]
    return res

def parabolic( x, *p ):
    a, mu, b = p
    res =   a*(x-mu)**2+b
    return res

#def sym_to_beam(xy, theta):
    #x = [point[0] for point in xy]
    #y = [point[1] for point in xy]
    #return (x*np.cos(theta)+y*np.sin(theta), -x*np.sin(theta)+y*np.cos(theta))

#def sym_to_beam(xy , theta):
#    x,y = xy
#    return (x*np.cos(theta)+y*np.sin(theta), -x*np.sin(theta)+y*np.cos(theta))
def sym_to_beam(xy, theta):
    xy = np.array(xy)  # Convert the input list to a NumPy array
    x = xy[:, 0]  # Extract the x-values
    y = xy[:, 1]  # Extract the y-values
    x_s = x * np.cos(theta) + y * np.sin(theta)
    y_s = -x * np.sin(theta) + y * np.cos(theta)
    return x_s, y_s

In [8]:
#scan_files = {'/eos/home-a/akallits/SWAN_projects/pico/run320_merged.root':'Pico'}
scan_files = {'./data/2023_April_h4/Run460_parameters.root':'Pico'} # Run 460 Pool 4 Picolarge Pad0 500/275 MCP3 pos -1570
#scan_files = {'/eos/home-a/akallits/SWAN_projects/pico/Run460_parameters.root':'Pico'} # Run 460 Pool 4 Picolarge Pad0 500/275 MCP3 pos -1570
#scan_files = {'/eos/home-a/akallits/SWAN_projects/pico/run323_merged.root':'Pico'}

In [9]:
def get_tree(run):
    rootfile = uproot.open(f"/eos/home-a/akallits/SWAN_projects/pico/Run{run}_parameters.root");
    #rootfile = uproot.open(f"/home/akallits/Desktop/JULY_RUNS/run{run}.root");
    key = rootfile.keys()
    print(key)

    tree = rootfile["Pico"]
    key = tree.keys()

    print(key)
    #rootfile.close()
    return tree

In [10]:
def get_df(tree, *indexes):
    dataframe = pd.DataFrame() # empty dataframe
    branches = []
    for i in indexes:
        try:
            branch = tree.keys()[i]
            branches.append(branch)
        except IndexError:
            print(f"Index {i} is out of range for the tree keys.")
    #branches =[tree.keys()[i] for i in indexes]

    for df, report in tree.iterate(branches, step_size='1 MB',library='pd', report = True):
        print(report)
        dataframe = pd.concat([dataframe,df], ignore_index=True)

    return dataframe

In [11]:
def rename_tree_branches(df,*col_names):
    l = len(col_names[0])  
    for i in range(l):
        df.rename(columns = {col_names[0][i]:col_names[1][i]}, inplace = True)
    
    return df

In [12]:
def store_new_tree(df):
    newfile = uproot.recreate("./example.root")
    newfile['tree'] = df
    newfile.close()

In [13]:
data =[]
run = [460]
for irun in run:
    tree = get_tree(irun)
    var_indexes = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39])
    data_single = get_df(tree,*var_indexes)
    old_names = ['MCP1_global_maximum_y','MCP1_e_charge', 'MCP1_all_charge','MCP1_sigmoid_timepoint','MM1_global_maximum_y','MM1_e_charge','MM1_all_charge','MM1_sigmoid_timepoint','POS0_track_hit', 'TrackChi2', 'NumberofTracks',  'SRSnumber']
    new_names = ['mcp1_gpeak','mcp1_qe', 'mcp1_qall','mcp1_t','mm1_gpeak','mm1_qe', 'mm1_qall','mm1_tfit','xy', 'chi2', 'track_flag',  'SRSnumber']
    names = [old_names,new_names]
    data_single = rename_tree_branches(data_single,*names)
    print(len(data_single))
    data.append(data_single)

# Concatenate the list of DataFrames into a single DataFrame
data = pd.concat(data)

##old code 

#for irun in run:
#     tree = get_tree(irun)
    
#     var_indexes = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38])
#     #for index in range(4):
#         #var_indexes = np.append(var_indexes,[index*10+8,index*10+9,index*10+12,index*10+14]) # choose which collums you need from the DF
#     #var_indexes = np.append(var_indexes,[67,65,66])
    
#     data_single = get_df(tree,*var_indexes)
    
#     old_names = ['MCP1_global_maximum_y','MCP1_all_charge','MCP1_sigmoid_timepoint','TrackChi2', 'NumberofTracks']
#     new_names = ['mcp1_gpeak','mcp1_qall','mcp1_t','chi2', 'track_flag']
#     names = [old_names,new_names]
#     data_single = rename_tree_branches(data_single,*names)
    
#     print(len(data_single))
#     data.append(data_single)

# data = pd.concat(data)


#print(data.head())
#print(len(data))

In [14]:
#Debug variables observation

column_number = data.iloc[0, 2]
print("Column Number:", column_number)


column_indices = data.columns
print(column_indices)

column_indices = list(range(len(data.columns)))
print(column_indices)


mcp_data =  data.iloc[:, 8]
print(mcp_data)


mm_data_charge =  data.iloc[:, 20].values
print(mm_data_charge)
len(mm_data_charge)


mm_data_charge_all =  data.iloc[:, 21].values
print(mm_data_charge_all)
len(mm_data_charge_all)

In [15]:
fig, axs = plt.subplots(figsize=(15, 10))
#for i in range(len(mcp_vs)):
axs.hist(data.mcp1_gpeak, bins=150, range=(0.0, 0.4), histtype='step', color='black', lw=2, label=f'MCP')
axs.hist(data.mm1_gpeak, bins=150, range=(0.0, 0.2), histtype='step', color='red', lw=2, label=f'MM pad')
axs.set_yscale('log')
axs.set_xlabel('Amplitude [V]')
axs.set_title('')
axs.legend()
plt.show()

In [16]:
fig, axs = plt.subplots(figsize=(15, 10))
#for i in range(len(mcp_qs)):
axs.hist(data.mcp1_qe, bins=100, range=(0.0, 8), histtype='step', color='black', lw=2, label=f'MCP')
axs.hist(data.mm1_qe, bins=100, range=(0.0, 8), histtype='step', color='red', lw=2, label=f'MM pad')
axs.set_yscale('log')
axs.set_xlabel('Charge [pC]')
axs.set_title('')
axs.legend()
plt.show()

In [17]:
fig, axs = plt.subplots(figsize=(15, 10))
#for i in range(len(mm_vs)):
axs.hist(data.mm1_gpeak, bins=100, range=(0.0, 0.175), histtype='step', color='red', lw=2, label=f'MM Pad')
axs.hist(data.mcp1_gpeak, bins=100, range=(0.0, 0.175), histtype='step', color='black', lw=2, label=f'MCP')
# axs.set_yscale('log')
axs.set_xlabel('Amplitude [V]')
axs.set_title('MM Pad')
axs.legend()
plt.show()

In [18]:
fig, axs = plt.subplots(figsize=(15, 10))
#for i in range(len(mm_qes)):
axs.hist(data.mm1_qe, bins=100, range=(0.0, 10), histtype='step', color="black", lw=2, label=f'MM Pad')
axs.hist(data.mcp1_qall, bins=100, range=(0.0, 10), histtype='step', color='red', lw=2, label=f'MCP')
axs.set_yscale('log')
axs.set_xlabel('e-Peak Charge [pC]')
axs.set_title('')
axs.legend()
plt.show()

In [19]:
fig, axs = plt.subplots(figsize=(15, 10))
#for i in range(len(mm_qalls)):
axs.hist(data.mm1_qall, bins=100, range=(0.0, 50), histtype='step', color='black', lw=2, label=f'MM Pad')
axs.hist(data.mcp1_qe, bins=100, range=(0.0, 10), histtype='step', color='red', lw=2, label=f'MCP')
axs.set_yscale('log')
axs.set_xlabel('Total Charge [pC]')
axs.set_title('MM Pads')
axs.legend()
plt.show()

In [20]:
fig, axs = plt.subplots(figsize=(15, 10))
#for i in range(len(mm_qalls)):
axs.hist(data.mm1_qall, bins=100, range=(0.0, 50), histtype='step', color='black', lw=2, label=f'MM Pad')
axs.hist(data.mm1_qe, bins=100, range=(0.0, 10), histtype='step', color='red', lw=2, label=f'MCP')
axs.set_yscale('log')
axs.set_xlabel('Total Charge [pC]')
axs.set_title('MM Pads')
axs.legend()
plt.show()

In [21]:
fig, axs = plt.subplots(figsize=(15, 10))

#for i in range(len(mm_vs)):
axs.scatter(data.mm1_gpeak, data.mm1_qe, s=0.6, color='black', label=f'MM Pad')
    
axs.set_xlabel('Amplitude [V]')
axs.set_ylabel('e-peak Charge [pC]')
axs.set_xlim(0, 0.18)
axs.set_ylim(0, 10)
axs.legend()

plt.show()

In [22]:
fig, axs = plt.subplots(figsize=(15, 10))

#for i in range(len(mm_vs)):
axs.scatter(data.mm1_gpeak, data.mm1_qall, s=0.6, color='black', label=f'MM Pad')
    
axs.set_xlabel('Amplitude [V]')
axs.set_ylabel('Total Charge [pC]')
axs.set_xlim(0, 0.18)
axs.set_ylim(0, 50)
axs.legend()

plt.show()

In [23]:
#xy = xy.values.reshape(-1, 2)  # Reshape into pairs of x-y values

## CALCULATIONS

In [24]:
track_flag = data.track_flag
valid_tracks = track_flag == 1

In [25]:
mcp_overflow = (data.mcp1_gpeak>0)*(data.mcp1_gpeak<=0.35)
chi2_cut = (data.chi2<=50)*(data.track_flag>0)
mcp_basic_cut = mcp_overflow*chi2_cut

In [26]:
print(data.xy)

In [27]:
if 'cc' in globals():
    del cc

cc = ROOT.TCanvas('cc', '', 800, 1000)

ROOT.gStyle.SetOptFit(1111)

# Check if a histogram with the name "xytrks" already exists
existing_hist = ROOT.gROOT.FindObject('xytrks')
if existing_hist:
    existing_hist.Delete()

#tracks multiplicity
xytrks = ROOT.TH2F("xytrks", f'RUN {run}: All tracks multiplicity', 100, 15, 80, 100, 10, 80)
xytrks.GetXaxis().SetTitle("x [mm]")
xytrks.GetYaxis().SetTitle("y [mm]")


n = len(data[mcp_basic_cut])
xy_values = data.xy[mcp_basic_cut].values
x_indices = range(0, len(xy_values), 3)
y_indices = range(1, len(xy_values), 3)
w = np.ones(n)

# Loop to fill the histogram
for i, x_idx in enumerate(x_indices):
    x_value = xy_values[x_idx]
    y_value = xy_values[y_indices[i]]
    xytrks.Fill(x_value, y_value, w[i])

# Clear out areas with low number of tracks (not enough statistics)
for i in range(xytrks.GetNbinsX() + 1):
    for j in range(xytrks.GetNbinsY() + 1):
        if xytrks.GetBinContent(i, j) < 1 :
            xytrks.SetBinContent(i, j, 0)

xytrks.SetTitle(f'RUN{run}: xy tracks with multiplicity > 1')
xytrks.Draw("colz")
cc.Draw()

In [28]:
cc.Draw()

In [29]:
# c = ROOT.TCanvas()

# xytrks = ROOT.TH2F("xytrks", f'RUN{run}: All tracks multiplicity', 100, 0, 50, 100, 0, 50)

# xytrks.GetXaxis().SetTitle("x")
# xytrks.GetYaxis().SetTitle("y")

# n = len(data[mcp_basic_cut])
# xy_values = data.xy[mcp_basic_cut].values
# x_indices = range(0, len(xy_values), 3)
# y_indices = range(1, len(xy_values), 3)
# w = np.ones(n)

# # Loop to fill the histogram
# for i, x_idx in enumerate(x_indices):
#     x_value = x[x_idx]
#     y_value = y[y_indices[i]]
#     xytrks.Fill(x_value, y_value, w[i])

# for i in range(xytrks.GetNbinsX() + 1):
#     for j in range(xytrks.GetNbinsY() + 1):
#         if xytrks.GetBinContent(i, j) < 15:
#             xytrks.SetBinContent(i, j, 0)

# xytrksW = ROOT.TH2F("xytrksW", f'RUN{run}: xy tracks weighted by MCP charge', 100, 0, 50, 100, 0, 50)
# xytrksW.GetXaxis().SetTitle("x [mm]")
# xytrksW.GetYaxis().SetTitle("y [mm]")

# w = data.iloc[:, 7][mcp_basic_cut].values

# xytrksW.FillN(n, x, y, w)

# xytrksW.Divide(xytrks)


In [30]:
import ctypes
c = ROOT.TCanvas()

xytrks = ROOT.TH2F("xytrks",f'RUN{run}: All tracks multiplicity',100,0,50,100,0,50)

xytrks.GetXaxis().SetTitle("x")
xytrks.GetYaxis().SetTitle("y")

n = len(data[mcp_basic_cut])
xy_values = data.xy[mcp_basic_cut].values
x = data.xy[mcp_basic_cut].values[::3]
print(x)
y = data.xy[mcp_basic_cut].values[1::3]
print(y)
w = np.ones(n)

# Convert NumPy arrays to ctypes arrays
x_arr = (ctypes.c_double * len(x))(*x)
y_arr = (ctypes.c_double * len(y))(*y)
w_arr = (ctypes.c_double * len(w))(*w)

# Now use these ctypes arrays in TH2::FillN
xytrks.FillN(n, x_arr, y_arr, w_arr)

#xytrks.FillN(n,x,y,w)

for i in range(xytrks.GetNbinsX()+1):
    for j in range(xytrks.GetNbinsY()+1):
        if xytrks.GetBinContent(i,j)<15:
            xytrks.SetBinContent(i,j,0)

xytrksW = ROOT.TH2F("xytrksW",f'RUN{run}: xy tracks weighted by MCP charge',100,0,50,100,0,50)
xytrksW.GetXaxis().SetTitle("x [mm]")
xytrksW.GetYaxis().SetTitle("y [mm]")

w = data.iloc[:, 7][mcp_basic_cut].values

xytrksW.FillN(n,x_arr, y_arr, w_arr)

xytrksW.Divide(xytrks)

In [31]:
#define the cuts
mm_basic_cut= []
#for index in range(4):

# mm_gpeak = f'MM1_global_maximum_y'
# mm_qall = f'MM1_all_charge'
# mm_qe = f'MM1_e_charge'
# mm_qfit = f'MM1_sigmoid_timepoint'
mm_overflow = (data.mm1_gpeak>0.)*(data.mm1_gpeak<0.36)*(data.mm1_qe>0)
qe_line = (data.mm1_qe>30.*data.mm1_gpeak)
#cut = np.logical_and(mm_overflow*mcp_basic_cut,qe_line)

mm_basic_cut = np.logical_and(mm_overflow*mcp_basic_cut,qe_line)

#mm_basic_cut.append(cut)

In [32]:
# we find the pad here!
    
xytrksW_mm = ROOT.TH2F(f'xytrksW - MM',f'RUN {run}: xy tracks weighted by PAD e-peak row charge',
                       100,0,50,100,0,50)
xytrksW_mm.GetXaxis().SetTitle("x")
xytrksW_mm.GetYaxis().SetTitle("y")

#selected_data = data.loc[mm_basic_cut[0]]
#print(selected_data)
n = len(data[mm_basic_cut])
print(n)

xy_values = data.xy[mm_basic_cut].values #selected_data.iloc[:, -4].values
print(xy_values)
x_indices = range(0, len(xy_values), 3)
x_indices_array = np.array(list(x_indices), dtype=float)
print(x_indices)
y_indices = range(1, len(xy_values), 3)
y_indices_array = np.array(list(y_indices), dtype=float)
print(y_indices)
xytrksW_mm.FillN(n,x_indices_array,y_indices_array,w)
    
xytrksW_mm.Divide(xytrks)
    

pad_hist = xytrksW_mm.Clone()
pad_hist.SetName(f'MMpad_hist')
pad_hist.SetTitle(f' MM-PAD')
for i in range(pad_hist.GetNbinsX()+1):
    for j in range(pad_hist.GetNbinsY()+1):
        if pad_hist.GetBinContent(i,j)<2.:
            pad_hist.SetBinContent(i,j,0)

## Display selected

In [33]:
v_limits = data.mm1_gpeak>0
qall_limits = (data.mm1_qall>0)*(data.mm1_qall<50)
qe_limits = (data.mm1_qe>0)*(data.mm1_qe<12)
qfit_limits = (data.mm1_qall>0)*(data.mm1_qall<12)

In [34]:
fig, axes = plt.subplots(3,2,figsize=(10,20))
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
fig.subplots_adjust(hspace=0.2)
fig.suptitle(f'RUN {run}: PAD distributions', fontsize=15, fontweight='bold')


ax = axes[0][0]
ax.hist(data.mm1_gpeak[v_limits], bins =100,color = 'black', histtype='step')
ax.hist(data.mcp1_gpeak[v_limits], bins =100,color = 'red', histtype='step')
ax.set_xlabel('e-peak amplitude [V]',loc='right')
ax.grid()
ax = axes[0][1]
ax.hist(data.mm1_gpeak[v_limits], bins =100,color = 'black', histtype='step')
ax.hist(data.mcp1_gpeak[v_limits], bins =100,color = 'red', histtype='step')

ax.set_xlabel('e-peak amplitude [V]',loc='right')
ax.grid()
ax.set_yscale('log')

ax = axes[1][0]
ax.hist(data.mm1_qall[qall_limits], bins =100, color = 'black', histtype='step')
ax.hist(data.mcp1_qall[qall_limits], bins =100, color = 'red', histtype='step')
ax.set_xlabel('Total raw charge [pC]',loc='right')
ax.grid()
ax = axes[1][1]
ax.hist(data.mm1_qall[qall_limits], bins =100, color = 'black', histtype='step')
ax.hist(data.mcp1_qall[qall_limits], bins =100, color = 'red', histtype='step')
ax.set_xlabel('Total raw charge [pC]',loc='right')
ax.set_yscale('log')
ax.grid()

ax = axes[2][0]
ax.hist(data.mm1_qe[qe_limits], bins =100, color = 'black', histtype='step')
ax.hist(data.mcp1_qe[qe_limits], bins =100, color = 'red', histtype='step')
ax.set_xlabel('E-peak raw charge [pC]',loc='right')
ax.grid()
ax = axes[2][1]
ax.hist(data.mm1_qe[qe_limits], bins =100, color = 'black', histtype='step')
ax.hist(data.mcp1_qe[qe_limits], bins =100, color = 'red', histtype='step')
ax.set_xlabel('E-peak raw charge [pC]',loc='right')
ax.set_yscale('log')
ax.grid()

# ax = axes[3][0]
# ax.hist(data[mm_qfit][qfit_limits], bins =100, color = 'black', histtype='step')
# ax.set_xlabel('Fitted charge [pC]',loc='right')
# ax.grid()
# ax = axes[3][1]
# ax.hist(data[mm_qfit][qfit_limits], bins =100, color = 'black', histtype='step')
# ax.set_xlabel('Fitted charge [pC]',loc='right')
# ax.set_yscale('log')
# ax.grid()


plt.show()

In [35]:
fig, axs = plt.subplots(2,sharex=True,figsize=(8,12))
fig.suptitle(f'RUN {run}: PAD Voltage Amplitude versus Charge ', fontsize=15, fontweight='bold')

axs[0].scatter(data.mm1_gpeak[v_limits], data.mm1_qall[v_limits], color='black',s=0.5)
axs[0].set_title('Qall ', fontsize=15, color= 'black', loc='left');
axs[0].set_ylim(0,130)

axs[1].scatter(data.mm1_gpeak[v_limits], data.mm1_qe[v_limits], color='black',s=0.5)
axs[1].set_title('Qe ', fontsize=15, color= 'black', loc='left');
axs[1].set_ylim(0,30)
#axs[1].set_xlim(0.004,0.12)
#axs[1].set_ylim(0,6.)
line = data.mm1_qe<=30.*data.mm1_gpeak
axs[1].scatter(data.mm1_gpeak[v_limits*line], data.mm1_qe[v_limits*line], color='red',s=0.5)

# axs[2].scatter(data.mm_gpeak][v_limits], data[mm_qfit][v_limits], color='black',s=0.5)
# axs[2].set_title('Qfit ', fontsize=15, color= 'black', loc='left');
# axs[2].set_ylim(0,30)
#axs[2].set_xlim(0.004,0.12)
#axs[2].set_ylim(0,6.)

fig.tight_layout()

In [36]:
# Assume xytrksW_mm is your histogram
is_empty = True
for i in range(1, xytrksW_mm.GetNbinsX() + 1):
    for j in range(1, xytrksW_mm.GetNbinsY() + 1):
        if xytrksW_mm.GetBinContent(i, j) != 0:
            is_empty = False
            break
    if not is_empty:
        break

if is_empty:
    print("Histogram xytrksW_mm is empty")
else:
    print("Histogram xytrksW_mm is not empty")

In [37]:
# Assume xytrksW_mm is your histogram
non_zero_bins = 0

for i in range(1, xytrksW_mm.GetNbinsX() + 1):
    for j in range(1, xytrksW_mm.GetNbinsY() + 1):
        if xytrksW_mm.GetBinContent(i, j) != 0:
            non_zero_bins += 1

print(f"Histogram xytrksW_mm has {non_zero_bins} non-zero bins")

In [38]:
cc.Draw()

In [39]:
# %jsroot on
# xytrksW_mm.Draw("lego2z")
# cc.Draw()

In [40]:
# xytrksW_mm.Draw("colz")
# cc.Draw()

In [41]:
# %jsroot off
# pad_hist.Draw("surf3")
# cc.Draw()

In [42]:
# # Assuming data.xy is a column containing tuples of x, y coordinates
# fig, axs = plt.subplots(figsize=(11, 11))
# axs.scatter([point[0] for point in data.xy], [point[1] for point in data.xy], s=1, color='black', label='All tracks')
# axs.set_xlim(0, 100)
# axs.set_ylim(0, 100)
# axs.legend(loc=3, markerscale=5)
# axs.set_xlabel('X [mm]')
# axs.set_ylabel('Y [mm]')
# plt.show()

In [43]:
fig, axs = plt.subplots(figsize=(15, 10))
axs.hist(data.chi2, bins=100, range=(0, 50), histtype='step', color='black', lw=2)

axs.set_yscale('log')
axs.set_xlabel(r'Track $\chi^2$')
plt.show()

In [44]:
#Debug variables observation II

print(mm_basic_cut)

len(mm_basic_cut)

type(mm_basic_cut)

x = data.xy[mcp_basic_cut].values
print(x)

x = data.xy[mcp_basic_cut].values[::3]
print(x)


y = data.xy[mcp_basic_cut].values[1::3]
print(y)


y_track = x[1::3]
print(y_track)


z_track = x[2::3]
print(z_track)

In [45]:
wqmcp = data.iloc[:,7][mcp_basic_cut].values
print(wqmcp)

In [46]:
len(wqmcp)

In [47]:
if 'cc' in globals():
    del cc

cc = ROOT.TCanvas('cc', '', 800, 1000)

ROOT.gStyle.SetOptFit(1111)

# Check if a histogram with the name "xytrks" already exists
existing_hist = ROOT.gROOT.FindObject('xytrks')
if existing_hist:
    existing_hist.Delete()

#tracks multiplicity
xytrks = ROOT.TH2F("xytrks", f'RUN {run}: All tracks multiplicity', 100, 15, 80, 100, 10, 80)
xytrks.GetXaxis().SetTitle("x [mm]")
xytrks.GetYaxis().SetTitle("y [mm]")


n = len(data[mcp_basic_cut])
xy_values = data.xy[mcp_basic_cut].values
x_indices = range(0, len(xy_values), 3)
y_indices = range(1, len(xy_values), 3)
w = np.ones(n)

# Loop to fill the histogram
for i, x_idx in enumerate(x_indices):
    x_value = xy_values[x_idx]
    y_value = xy_values[y_indices[i]]
    xytrks.Fill(x_value, y_value, w[i])

# Clear out areas with low number of tracks (not enough statistics)
for i in range(xytrks.GetNbinsX() + 1):
    for j in range(xytrks.GetNbinsY() + 1):
        if xytrks.GetBinContent(i, j) < 1 :
            xytrks.SetBinContent(i, j, 0)

xytrks.SetTitle(f'RUN{run}: xy tracks with multiplicity > 1')
xytrks.Draw("colz")
cc.Draw()

In [48]:
cc.Draw()

In [49]:
#root canvas for display
c = ROOT.TCanvas('c1', '', 800, 1000)

ROOT.gStyle.SetOptFit(1111)

#tracks multiplicity
xytrks = ROOT.TH2F("xytrks", f'RUN {run}: All tracks multiplicity', 100, -150, 250, 100, -150, 250)
xytrks.GetXaxis().SetTitle("x [mm]")
xytrks.GetYaxis().SetTitle("y [mm]")

n = len(data[mcp_basic_cut])
xy_values = data.xy[mcp_basic_cut].values
x_indices = range(0, len(xy_values), 3)
y_indices = range(1, len(xy_values), 3)
w = np.ones(n)

# Loop to fill the histogram
for i, x_idx in enumerate(x_indices):
    x_value = xy_values[x_idx]
    y_value = xy_values[y_indices[i]]
    xytrks.Fill(x_value, y_value, w[i])

# Clear out areas with low number of tracks (not enough statistics)
for i in range(xytrks.GetNbinsX() + 1):
    for j in range(xytrks.GetNbinsY() + 1):
        if xytrks.GetBinContent(i, j) < 1:
            xytrks.SetBinContent(i, j, 0)

xytrks.SetTitle(f'RUN{run}: xy tracks with multiplicity >1')
xytrks.Draw("colz")
c.Draw()

In [50]:
#root canvas for display
c1 = ROOT.TCanvas('c2', '', 800, 1000)

ROOT.gStyle.SetOptFit(1111)

#tracks multiplicity
xytrks = ROOT.TH2F("xytrks", f'RUN {run}: All tracks multiplicity', 100, 35, 60, 100, 30, 60)
xytrks.GetXaxis().SetTitle("x [mm]")
xytrks.GetYaxis().SetTitle("y [mm]")

n = len(data[mcp_basic_cut])
xy_values = data.xy[mcp_basic_cut].values
x_indices = range(0, len(xy_values), 3)
y_indices = range(1, len(xy_values), 3)
w = np.ones(n)

# Loop to fill the histogram
for i, x_idx in enumerate(x_indices):
    x_value = xy_values[x_idx]
    y_value = xy_values[y_indices[i]]
    xytrks.Fill(x_value, y_value, w[i])

# Clear out areas with low number of tracks (not enough statistics)
for i in range(xytrks.GetNbinsX() + 1):
    for j in range(xytrks.GetNbinsY() + 1):
        if xytrks.GetBinContent(i, j) < 5 :
            xytrks.SetBinContent(i, j, 0)

xytrks.SetTitle(f'RUN{run}: xy tracks with multiplicity > 5')
xytrks.Draw("colz")
c1.Draw()

In [51]:
c1.Draw()

In [52]:
w = data.mcp1_qe[mcp_basic_cut].values

In [53]:
print(w)
len(w)
n = len(data[mcp_basic_cut])
type(mcp_basic_cut)
print(n)

In [54]:
#root canvas for display
c3 = ROOT.TCanvas('c3', '', 800, 1000)
c3.Divide(1,2)

xytrksW = []
ROOT.gStyle.SetOptFit(1111)

# Loop for the two MCP histograms
#for which_mcp in range(4):
for which_mcp in range(1):
    h = ROOT.TH2F(f'xyW_mcp{which_mcp+1}', f'RUN {run}: xy tracks weighted by MCP (in pool {4+which_mcp}) charge',
                 100, 35, 60, 100, 30, 60)

    h.GetXaxis().SetTitle("x [mm]")
    h.GetYaxis().SetTitle("y [mm]")

    #w = data.iloc[:, 8 * which_mcp + 1][mcp_basic_cut].values
    w = data.iloc[:, 7][mcp_basic_cut].values
    
    # Loop to fill the histogram
    for i, x_idx in enumerate(x_indices):
        x_value = xy_values[x_idx]
        y_value = xy_values[y_indices[i]]
        h.Fill(x_value, y_value, w[i])

    h.Divide(xytrks)
    xytrksW.append(h)

#     c3.cd(2 * which_mcp + 1)
#     xytrksW[which_mcp].Draw("lego2z")
#     c3.cd(2 * which_mcp + 2)
#     xytrksW[which_mcp].Draw("colz")

    c3.cd(2 * which_mcp + 1)
    xytrksW[which_mcp].Draw("surf3")
    c3.cd(2 * which_mcp + 2)
    xytrksW[which_mcp].Draw("lego2z")

c3.Draw()

In [55]:
c3.Draw()

In [56]:
c3.Clear()
c3.Divide(2,2)
for i in range(1):
    c3.cd(1)
    xytrksW[which_mcp].Draw("colz")#"lego2z"
    c3.cd(2)
    xytrksW[which_mcp].Draw("surf3")#"lego2z"
    c3.cd(3)
    xytrksW[which_mcp].Draw("lego2z")
c3.Update()
c3.Draw()

In [57]:
#selected_data = data.loc[mm_basic_cut]
#print(selected_data)
#n = len(selected_data)
n = len(data[mm_basic_cut])
print(n)
xy_values = data.xy[mm_basic_cut].values#xy_values = selected_data.iloc[:, -4].values
print(xy_values)
x_indices = range(0, len(xy_values), 3)
x_indices_array = np.array(list(x_indices), dtype=float)
print(x_indices)
y_indices = range(1, len(xy_values), 3)
y_indices_array = np.array(list(y_indices), dtype=float)
print(y_indices)
# print(xy_values)
# x_indices = range(0, len(xy_values), 3)
# print(x_indices)
# y_indices = range(1, len(xy_values), 3)
# print(y_indices)

In [58]:
print(data.mm1_qe[mm_basic_cut].values)
print(len(data.mm1_qe[mm_basic_cut].values))

In [59]:
pad_hist = xytrksW_mm.Clone()
pad_hist.SetName(f'MMpad_hist')
pad_hist.SetTitle(f' MM-PAD')
for i in range(pad_hist.GetNbinsX()+1):
    for j in range(pad_hist.GetNbinsY()+1):
        if pad_hist.GetBinContent(i,j)<2.:
            pad_hist.SetBinContent(i,j,0)

In [60]:
# Find the pads here!
xytrksW_mm = []  # Weighted histograms
pad_hist = []
#x = np.zeros(n, dtype=np.double)
#y = np.zeros(n, dtype=np.double)

#for index in range(1):
#pad = index + 1

h1 = ROOT.TH2F(f'xytrksW-pad0', f'RUN{run}: xy tracks weighted by PADs e-peak row charge',
                   100, 35, 60, 100, 30, 60)
    
h1.GetXaxis().SetTitle("x [mm]")
h1.GetYaxis().SetTitle("y [mm]")

# Apply the boolean mask to the DataFrame using .loc
#selected_data = data.loc[mm_basic_cut]

# n = len(selected_data)
# xy_values = selected_data.iloc[:, -4].values
# x_indices = range(0, len(xy_values), 3)
# y_indices = range(1, len(xy_values), 3)
#w = selected_data.iloc[:, 20].values
w = data.mm1_qe[mm_basic_cut].values

#w = selected_data.iloc[20].values
if len(w) > 0:
    # 'w' is not empty, so you can proceed with filling the histogram
    #h1.FillN(n, x, y, w)
    h1.FillN(n, xy_values[x_indices], xy_values[y_indices], w)

else:
    print("Warning: 'w' array is empty.")
#h1.FillN(n, x, y, w)

h1.Divide(xytrks)

xytrksW_mm.append(h1)

h2 = ROOT.TH2F(f'pad0_hist', f'PAD0',100, 35, 60, 100, 30, 60)

h2 = h1.Clone()
#h2.Copy(h1)

# Print information about h2 before filling
print(f'Before filling - pad0_hist information:')
print(f'Number of entries: {h2.GetEntries()}')
print(f'Integral: {h2.Integral()}')
print(f'Mean: {h2.GetMean()}')
print(f'RMS: {h2.GetRMS()}')

for i in range(h2.GetNbinsX() + 1):
    for j in range(h2.GetNbinsY() + 1):
        if h2.GetBinContent(i, j) < 1.:
            h2.SetBinContent(i, j, 0)

pad_hist.append(h2)

In [61]:
c.Clear()
c.Divide(2,2)
for i in range(1):
    c.cd(1)
    xytrksW_mm[0].Draw("colz")#"lego2z"
    c.cd(2)
    xytrksW_mm[0].Draw("surf3")#"lego2z"
    c.cd(3)
    xytrksW_mm[0].Draw("lego2z")#"lego2z"
c.Update()
c.Draw()

In [62]:
c.Draw()

In [63]:
c.Clear()
c.Divide(2,2)
for i in range(1):
    c.cd(1)
    h2.Draw("colz")#"lego2z"
    c.cd(2)
    h2.Draw("surf3")#"lego2z"
    c.cd(3)
    h2.Draw("lego2z")#"lego2z"
c.Update()
c.Draw()

In [64]:
c.Draw()

In [65]:
c.Clear()
c.Divide(1,2)
for i in range(1):
    #c.cd(1)
    #xytrksW_mm[0].Draw("colz")#"lego2z"
    c.cd(1)
    xytrksW_mm[0].Draw("surf3")#"lego2z"
    c.cd(2)
    xytrksW_mm[0].Draw("lego2z")#"lego2z"
c.Update()
c.Draw()

In [66]:
c.Draw()

In [67]:
# Find the pads here!
xytrksW_mm_qall = []  # Weighted histograms
pad_hist_qall = []
#x = np.zeros(n, dtype=np.double)
#y = np.zeros(n, dtype=np.double)

#for index in range(1):
#pad = index + 1

h1_qall = ROOT.TH2F(f'xytrksW-pad0_qall', f'RUN{run}: xy tracks weighted by PADs total charge',
                   100, 35, 60, 100, 30, 60)
    
h1_qall.GetXaxis().SetTitle("x [mm]")
h1_qall.GetYaxis().SetTitle("y [mm]")

# Apply the boolean mask to the DataFrame using .loc
#selected_data = data.loc[mm_basic_cut]

# n = len(selected_data)
# xy_values = selected_data.iloc[:, -4].values
# x_indices = range(0, len(xy_values), 3)
# y_indices = range(1, len(xy_values), 3)
#w = selected_data.iloc[:, 20].values
w_qall = data.mm1_qall[mm_basic_cut].values

#w = selected_data.iloc[20].values
if len(w) > 0:
    # 'w' is not empty, so you can proceed with filling the histogram
    #h1.FillN(n, x, y, w)
    h1.FillN(n, xy_values[x_indices], xy_values[y_indices], w)

else:
    print("Warning: 'w' array is empty.")
#h1.FillN(n, x, y, w)

h1_qall.Divide(xytrks)

xytrksW_mm_qall.append(h1_qall)

h2_qall = ROOT.TH2F(f'pad0_hist_qall', f'PAD0',100, 35, 60, 100, 30, 60)

h2_qall = h1_qall.Clone()
#h2.Copy(h1)

# Print information about h2 before filling
print(f'Before filling - pad0_hist information:')
print(f'Number of entries: {h2_qall.GetEntries()}')
print(f'Integral: {h2_qall.Integral()}')
print(f'Mean: {h2_qall.GetMean()}')
print(f'RMS: {h2_qall.GetRMS()}')

for i in range(h2_qall.GetNbinsX() + 1):
    for j in range(h2_qall.GetNbinsY() + 1):
        if h2_qall.GetBinContent(i, j) < 1.:
            h2_qall.SetBinContent(i, j, 0)

pad_hist.append(h2_qall)

## Pad position

In [68]:
from scipy.optimize import minimize

In [69]:
#set initial data
xdata = np.zeros(4)
ydata = np.zeros(4)
sigmax = np.ones(4)
sigmay = np.ones(4)

In [90]:
# Print each element to understand its structure
for i, element in enumerate(pad_hist):
    print(f"Element {i}: {element}")

In [86]:
pad_hist

In [87]:
#for index in range(4):
for index in range(1):
    
    #pad centers initialised by mean of weighted hist
    xdata[index] = pad_hist[index].GetMean(1)
    ydata[index] = pad_hist[index].GetMean(2)
    #sigmas
    sumx = 0
    sumy = 0
    sumq = 0

    for i in range(pad_hist[index].GetNbinsX()):
        for j in range(pad_hist[index].GetNbinsY()):
            delta_bin = pad_hist[index].GetBinError(i,j)
            xi = pad_hist[index].GetXaxis().GetBinCenter(i)
            yi = pad_hist[index].GetYaxis().GetBinCenter(j)
            sumx += delta_bin**2*(xi-xdata[index])**2
            sumy += delta_bin**2*(yi-ydata[index])**2
            sumq += pad_hist[index].GetBinContent(i,j)

    sigmax[index] = np.sqrt(sumx/sumq)
    sigmay[index] = np.sqrt(sumy/sumq)

In [89]:
# Print each element to understand its structure
for i, element in enumerate(xytrksW):
    print(f"Element {i}: {element}")

In [91]:
# minimize chi2 to get initial values of params 
#in case you have several pools you need to take the corresponding MCP, so chech which MCP version at constraint_fit.py code
# x0i = - (xytrksW[0].GetMean(1)+xytrksW[1].GetMean(1))/2
# y0i = - (xytrksW[0].GetMean(2)+xytrksW[1].GetMean(2))/2
x0i = - (xytrksW[0].GetMean(1))/2
y0i = - (xytrksW[0].GetMean(2))/2

ui = 0.
initial_guess = [x0i,y0i,ui]
boundaries = [(-60.,0.),(-60.,0.),(-np.pi,np.pi)]

In [92]:
result = minimize(chi2, initial_guess,bounds=boundaries)
if result.success:
    x0i,y0i,ui = result.x
    print(result)
else:
    raise ValueError(result.message) 

In [93]:
xc_step1 = np.zeros(4)
yc_step1 = np.zeros(4)
xc_er = np.zeros(4)
yc_er = np.zeros(4)
xc_step2 = np.ones(4)
yc_step2 = np.ones(4)

In [94]:
#for index in range(4):
for index in range(1):
    xc_step1[index], yc_step1[index] = beam_coords(index,result.x)

In [95]:
xy_values = data.xy.values
print(xy_values)

In [96]:
x = range(0, len(xy_values), 3)

y = range(1, len(xy_values), 3)

print(x,y)

In [98]:
for alg_steps in range(20): #if in 20 steps not converged.. terminates
    #1st step of the algorithm
    h2 =[None] * 4
    projX =[None] * 4
    projY =[None] * 4

    #for index in range(4):
    for index in range(1):   
        pad = index +1      

        h1 = ROOT.TH2F('',"",
                   28,xc_step1[index]-7.,xc_step1[index]+7.,28,yc_step1[index]-7.,yc_step1[index]+7.)
        h2[index] = ROOT.TH2F('',f"PAD {[index]} distribution in Local frame",
                   28,xc_step1[index]-7.,xc_step1[index]+7.,28,yc_step1[index]-7.,yc_step1[index]+7.)

        h2[index].GetXaxis().SetTitle(f'x symmetry axis')
        h2[index].GetYaxis().SetTitle(f'y symmetry axis')
        #update estimations about centers in beam frame
        xc_step1[index], yc_step1[index] = beam_coords(index,result.x)

#         x = data.iloc[:,-4].values
#         y = data.iloc[:,-3].values
        xy_values = data.xy.values
        x = range(0, len(xy_values), 3)
        y = range(1, len(xy_values), 3)
        xsym, ysym = sym_to_beam((x,y),-ui)
        #circle_cut = np.sqrt((data.iloc[:,-4]-xc)**2 + (data.iloc[:,-3]-yc)**2)<7.
        #square_cut = np.logical_and(np.abs(data.iloc[:,-4]-xc)<7., np.abs(data.iloc[:,-3]-yc)<7.)
        circle_cut = np.sqrt((np.abs(xsym-xc_step1[index])**2 + np.abs(ysym-yc_step1[index])**2))<6.
        #square_cut = np.logical_and(np.abs(xsym-xc_step1[index])<6., np.abs(ysym-yc_step1[index])<6.)
        
        #n = len(data[mm_basic_cut[index]*square_cut])
        n = len(data[mm_basic_cut[index]*circle_cut])

        # x = data.iloc[:,-4][mm_basic_cut[index]*square_cut].values
        # y = data.iloc[:,-3][mm_basic_cut[index]*square_cut].values
    
        # xsym, ysym = sym_to_beam((x,y),-ui)
#         xsym = xsym[mm_basic_cut[index]*square_cut]
#         ysym = ysym[mm_basic_cut[index]*square_cut]
        xsym = xsym[mm_basic_cut[index]*circle_cut]
        ysym = ysym[mm_basic_cut[index]*circle_cut]
        
        w = np.ones(n)
        h1.FillN(n,xsym,ysym,w)
        
        #w = data.iloc[:,7+index*4][mm_basic_cut[index]*square_cut].values
        w = data.mm1_qe[mm_basic_cut[index]*circle_cut].values
        h2[index].FillN(n,xsym,ysym,w)
    
        h2[index].Divide(h1)
        #clear out
        for i in range(h2[index].GetNbinsX()+1):
            for j in range(h2[index].GetNbinsY()+1):
                if h2[index].GetBinContent(i,j)<2.:
                    h2[index].SetBinContent(i,j,0)

        
        #projections to symmetry axis
        projX[index] = h2[index].ProjectionX(f'PAD{index}_px ')
        projX[index].SetTitle(f'PAD {[index]} x projection (Local Frame)')
        projX[index].GetXaxis().SetTitle(f'x symmetry axis')
        projX[index].Scale(1./projX[index].Integral())

        projY[index] = h2[index].ProjectionY(f'PAD{pad}_py ')
        projY[index].SetTitle(f'PAD {[index]} y projection (Local Frame)')
        projY[index].GetXaxis().SetTitle(f'y symmetry axis')
        projY[index].Scale(1./projY[index].Integral())


    c.Clear()
    c.Divide(2,2)
    #for i in range(4):
    for i in range(1):
        c.cd(i+1)
        h2[3-i].Draw("surf3")#"lego2z"
    c.Update()
    #input('Enter to continue ')

    #for index in range(4):
    for index in range(1):   
        pad = index +1
        #fit
        mean = projX[index].GetMean()
        projX[index].SetAxisRange(mean-10,mean+10, 'X')
        fitf = ROOT.TF1("fitf", weird_gaussian,mean-6,mean+6,4)
        #fitf = ROOT.TF1("fitf", parabola,mean-6,mean+6,3)
    
        binmax = projX[index].GetMaximumBin()
        maximum = projX[index].GetXaxis().GetBinCenter(binmax)
        par = np.array([maximum,mean-2.,mean+2.,projX[index].GetStdDev()])
        #par = np.array([-projX[index].GetStdDev(),mean,maximum])
        fitf.SetParameters(par)

        fit_res = projX[index].Fit(fitf,"RSQ")

        mux = (fit_res.Parameter(1)+fit_res.Parameter(2))/2
        xc_er[index] = np.sqrt(fit_res.ParError(1)**2+fit_res.ParError(1)**2) #approximately
        print(f'fit X pars errors: {xc_er[index]}')
        
        # old technique with projections

        # size = projX[index].GetNbinsX()
        # bins_centers = np.zeros(size)
        # bins_counts = np.zeros(size)
        # bins_errors = np.zeros(size)

        # test = projX[index].Clone()
        # for i in range(size):
        #     bins_centers[i] = projX[index].GetXaxis().GetBinCenter(i)
        #     bins_counts[i] = projX[index].GetBinContent(i)
        #     bins_errors[i] = projX[index].GetBinError(i)
        #     if(bins_counts[i]>0.048):
        #         bins_errors[i] = 0.5
        #     test.SetBinError(i,bins_errors[i])
        
        
        # fit_res = test.Fit(fitf,"RSQ")
        # fitf.GetParameters(par)

        # mux = fit_res.Parameter(1)
        # xc_er[index] = fit_res.ParError(1)
        # print(f'fit X pars errors: {xc_er[index]}')
        
        # plt.subplot(2, 2, index + 1)
        # plt.errorbar(bins_centers, bins_counts, fmt='o',markersize=2.,xerr=0.25,color='black')
        # plt.errorbar(bins_centers[bins_errors<0.5], bins_counts[bins_errors<0.5],
        #     yerr= bins_errors[bins_errors<0.5], xerr=0.25,fmt='o',markersize=2.,color='black',
        #     label=f'PAD {pad_name[index]}')

        # x = np.linspace(bins_centers[bins_counts>0][0],bins_centers[bins_counts>0][6])
        # plt.plot(x,parabolic(x,*par),'r--',label=r'fit: $\alpha\cdot(x-\mu)^2+\beta$')

        # x = np.linspace(bins_centers[bins_counts>0][-8]+0.25,bins_centers[bins_counts>0][-1]-0.25)
        # plt.plot(x,parabolic(x,*par),'r--')

        # plt.xlim(mean-6.8,mean+6.8)
        # plt.xlabel('x symmetry axis [mm]')
        # plt.legend()

        # popt, pcov = curve_fit(weirdo_gaussian, bins_centers, bins_counts, p0= par)

        # mux = (popt[1]+popt[2])/2
        # xc_er[index] = np.sqrt(pcov[1,1]**2+pcov[2,2]**2)/2 #approximately
        # print(f'fit X pars errors: {xc_er[index]}')

        #same for y
        mean = projY[index].GetMean()
        projY[index].SetAxisRange(mean-10,mean+10, 'X')
        fitf = ROOT.TF1("fitf", weird_gaussian,mean-6,mean+6,4)
        #fitf = ROOT.TF1("fitf", parabola,mean-6,mean+6,3)
    
        binmax = projY[index].GetMaximumBin()
        maximum = projY[index].GetXaxis().GetBinCenter(binmax)
        par = np.array([maximum,mean-2.,mean+2.,projY[index].GetStdDev()])
        #par = np.array([-projY[index].GetStdDev(),mean,maximum])
        fitf.SetParameters(par)
    
        fit_res = projY[index].Fit(fitf,"RSQ")

        muy = (fit_res.Parameter(1)+fit_res.Parameter(2))/2
        yc_er[index] = np.sqrt(fit_res.ParError(1)**2+fit_res.ParError(1)**2)
        print(f'fit Y pars errors: {yc_er[index]}')

        # size = projY[index].GetNbinsX()
        # bins_centers = np.zeros(size)
        # bins_counts = np.zeros(size)
        # bins_errors = np.zeros(size)

        # test = projY[index].Clone()
        # for i in range(size):
        #     bins_centers[i] = projY[index].GetXaxis().GetBinCenter(i)
        #     bins_counts[i] = projY[index].GetBinContent(i)
        #     bins_errors[i] = projY[index].GetBinError(i)
        #     if(bins_counts[i]>0.05):
        #         bins_errors[i] = 0.5
        #     test.SetBinError(i,bins_errors[i])
        
        # fit_res = test.Fit(fitf,"RSQ")
        # fitf.GetParameters(par)

        # muy = fit_res.Parameter(1)
        # yc_er[index] = fit_res.ParError(1)
        # print(f'fit Y pars errors: {yc_er[index]}')

        # plt.subplot(2, 2, index + 1)
        # plt.errorbar(bins_centers, bins_counts, fmt='o',markersize=2.,xerr=0.25,color='black')
        # plt.errorbar(bins_centers[bins_errors<0.5], bins_counts[bins_errors<0.5],
        #     yerr= bins_errors[bins_errors<0.5], xerr=0.25,fmt='o',markersize=2.,color='black',
        #     label=f'PAD {pad_name[index]}')

        # x = np.linspace(bins_centers[bins_counts>0][0]+0.25,bins_centers[bins_counts>0][8])
        # plt.plot(x,parabolic(x,*par),'r--',label=r'fit: $\alpha\cdot(x-\mu)^2+\beta$')

        # x = np.linspace(bins_centers[bins_counts>0][-8]+0.25,bins_centers[bins_counts>0][-1]+0.125)
        # plt.plot(x,parabolic(x,*par),'r--')

        # plt.xlim(mean-6.8,mean+6.8)
        # plt.xlabel('y symmetry axis [mm]')
        # plt.legend()

        # popt, pcov = curve_fit(weirdo_gaussian, bins_centers, bins_counts, p0= par)
        # muy = (popt[1]+popt[2])/2
        # yc_er[index] = np.sqrt(pcov[1,1]**2+pcov[2,2]**2)/2 #approximately
        # print(f'fit Y pars errors: {yc_er[index]}')

        #independent est. for pad centers (beam frame)
        mux = h2[index].GetMean(1)
        muy = h2[index].GetMean(2)
        xc_step1[index], yc_step1[index] = sym_to_beam((mux,muy),ui)

         

   # display projections
    # c.Clear()
    # c.Divide(2,2)
    # for i in range(4):
    #     c.cd(i+1)
    #     projX[i].Draw()
    # c.Update()
    # input('Enter to continue ')
    

    # c.Clear()
    # c.Divide(2,2)
    # for i in range(4):
    #     c.cd(i+1)
    #     projY[i].Draw()
    # c.Update()
    # input('Enter to continue ')

    #update data pnts (input in minimization) according to step1
    xdata = xc_step1
    ydata = yc_step1

    #udate sigmas
    #for index in range(4):
    for index in range(1):

        #sigmas
        sumx = 0
        sumy = 0
        sumq = 0
        

        for i in range(pad_hist[index].GetNbinsX()):
            for j in range(pad_hist[index].GetNbinsY()):
                delta_bin = pad_hist[index].GetBinError(i,j)
                xi = pad_hist[index].GetXaxis().GetBinCenter(i)
                yi = pad_hist[index].GetYaxis().GetBinCenter(j)
                sumx += delta_bin**2*(xi-xdata[index])**2
                sumy += delta_bin**2*(yi-ydata[index])**2
                sumq += pad_hist[index].GetBinContent(i,j)

        sigmax[index] = np.sqrt(sumx/sumq)
        sigmay[index] = np.sqrt(sumy/sumq)
    
    # minimize chi2 to get param values
    initial_guess = [x0i,y0i,ui]

    result = minimize(chi2, initial_guess)
    if result.success:
        x0i,y0i,ui = result.x
        print(result)
    else:
        raise ValueError(result.message) 

    #estimations of step2
    #for index in range(4):
    for index in range(1):
        xc_step2[index], yc_step2[index] = beam_coords(index,result.x)

    #update(xc_step1,yc_step1)
    print(f'step1: x_center ={xc_step1} ')
    print(f'step1: y_center ={yc_step1} ')
    print(f'step2: x_center ={xc_step2} ')
    print(f'step2: y_center ={yc_step2} ')

    ftol = 1e-06 #default
    for i in range(len(result.x)):
        hess_inv_i = result.hess_inv[i,i]
        uncertainty_i = np.sqrt(max(1, abs(result.fun)) * ftol * hess_inv_i)
        print('p^{0} = {1:12.4e} ± {2:.1e}'.format(i, result.x[i], uncertainty_i))


    x_check = np.abs(xc_step1-xc_step2)<=np.abs(0.05) # errors put of 0.05-0.04 for converge
    y_check= np.abs(yc_step1-yc_step2)<=np.abs(0.04)

    if x_check[0]==True and y_check[0]==True:
        if np.all(x_check==x_check[0]) and np.all(y_check==y_check[0]):
            print('--------------------------------------')
            print('CONVERGENCE REACHED')
            print('--------------------------------------')
            centers = np.empty((2,4))
            centers[0]=xc_step2
            centers[1]=yc_step2
            print(centers)
            #with open(f'/home/akallits/Desktop/JULY_RUNS/run_{run[0]}_{run[1]}_info/pad_centers.txt', 'w') as f: #save results to txt
            with open(f'/eos/home-a/akallits/SWAN_projects/pico/run_{run[0]}_{run[1]}_info/pad_centers.txt', 'w') as f: #save results to txt
                np.savetxt(f,centers, delimiter=" ")
            exit(0)

print('--------------------------------------')
print('TOO MANY ITERATIONS WITHOUT CONVERGING')
print('--------------------------------------')

In [99]:
# Assuming `data` is your DataFrame
print(f"Length of DataFrame: {len(data)}")

# Check the length and shape of mm_basic_cut
print(f"Length of mm_basic_cut: {len(mm_basic_cut)}")
print(f"Shape of mm_basic_cut: {mm_basic_cut.shape}")
print(mm_basic_cut)

# Check the length and shape of circle_cut
print(f"Length of circle_cut: {len(circle_cut)}")
print(f"Shape of circle_cut: {circle_cut.shape}")
print(circle_cut)

# Ensure that the index you are using is within the bounds of mm_basic_cut
if index >= len(mm_basic_cut):
    raise IndexError(f"Index {index} is out of bounds for mm_basic_cut with length {len(mm_basic_cut)}")

# Check the specific indexer used in the expression
print(f"Value of mm_basic_cut at index {index}: {mm_basic_cut[index]}")

# Perform the element-wise multiplication
combined_indexer = mm_basic_cut[index] * circle_cut

# Check the length and shape of the combined indexer
print(f"Length of combined_indexer: {len(combined_indexer)}")
print(f"Shape of combined_indexer: {combined_indexer.shape}")
print(combined_indexer)

# Now apply the combined indexer to the DataFrame
n = len(data[combined_indexer])
print(f"Number of selected rows: {n}")


# constraint fit

In [70]:
#set initial data
xdata = np.zeros(4) #(4)
ydata = np.zeros(4)
sigmax = np.ones(4)
sigmay = np.ones(4)

In [71]:
len(xytrksW[0])

In [72]:
x0i = - (xytrksW[0].GetMean(1))/2

In [73]:
print(x0i)

In [74]:
y0i = - (xytrksW[0].GetMean(2))/2 
print(y0i)

In [75]:
#for index in range(4):
#     for i in range(pad_hist[index].GetNbinsX()):
#         for j in range(pad_hist[index].GetNbinsY()):
#             delta_bin = pad_hist[index].GetBinError(i,j)
#             xi = pad_hist[index].GetXaxis().GetBinCenter(i)
#             yi = pad_hist[index].GetYaxis().GetBinCenter(j)
#             sumx += delta_bin**2*(xi-xdata[index])**2
#             sumy += delta_bin**2*(yi-ydata[index])**2
#             #sumx += max(0, delta_bin ** 2 * (xi - xdata[index]) ** 2)
#             #sumy += max(0, delta_bin ** 2 * (yi - ydata[index]) ** 2)
#             sumq += pad_hist[index].GetBinContent(i,j)


        # ...

#for index in range(1):
index = 0
xdata[index] = h2.GetMean(1)
ydata[index] = h2.GetMean(2)
#sigmas
sumx = 0
sumy = 0
sumq = 0

for i in range(h2.GetNbinsX()):
    for j in range(h2.GetNbinsY()):
        delta_bin = h2.GetBinError(i, j)
        xi = h2.GetXaxis().GetBinCenter(i)
        yi = h2.GetYaxis().GetBinCenter(j)
        sumx += delta_bin**2 * (xi - xdata[index])**2
        sumy += delta_bin**2 * (yi - ydata[index])**2
        sumq += pad_hist[index].GetBinContent(i, j)



if sumq != 0:
    sigmax[index] = np.sqrt(sumx/sumq)
    sigmay[index] = np.sqrt(sumy/sumq)
else:
    # Handle the case where sumq is zero, e.g., set sigmax and sigmay to some default values.
#     sigmax[index] = pad_hist[index].GetRMS(1)
#     sigmay[index] = pad_hist[index].GetRMS(2)
      sigmax[index] = h2.GetRMS(1)
      sigmay[index] = h2.GetRMS(2)
    #sigmax[index] = np.sqrt(sumx/sumq)
    #sigmay[index] = np.sqrt(sumy/sumq)

# minimize chi2 to get initial values of params
#x0i = - (xytrksW[0].GetMean(1)+xytrksW[1].GetMean(1))/2
#y0i = - (xytrksW[0].GetMean(2)+xytrksW[1].GetMean(2))/2 

#when you have only one scope to analyze

x0i = - (xytrksW[0].GetMean(1))/2
y0i = - (xytrksW[0].GetMean(2))/2 

In [76]:
print(x0i)
print(y0i)

In [77]:
print(xdata)

In [78]:
ui = 0.
initial_guess = [x0i,y0i,ui]
boundaries = [(-50.,0.),(-50.,0.),(-np.pi,np.pi)]

In [79]:
result = minimize(chi2, initial_guess,bounds=boundaries)

In [80]:
print(result)

In [81]:
if result.success:
    x0i,y0i,ui = result.x
    print(result)
else:
    raise ValueError(result.message)

In [82]:
xc_step1 = np.zeros(4)
yc_step1 = np.zeros(4)
xc_er = np.zeros(4)
yc_er = np.zeros(4)
xc_step2 = np.ones(4)
yc_step2 = np.ones(4)

#for index in range(4):
for index in range(1):
    xc_step1[index], yc_step1[index] = beam_coords(index,result.x)

In [83]:
data_reset = data.reset_index(drop=True)

In [84]:
print(selected_data)

In [ ]:
print(xc_step1[0])

In [ ]:
selected_data = data_reset[(mm_basic_cut[0])]

In [ ]:
print(selected_data)

In [ ]:
print(mm_basic_cut[0])

In [ ]:
print(selected_data)

In [ ]:
# mm_basic_cut[0] = np.full(len(data_reset), False)

In [ ]:
print(mm_basic_cut)


print(w)

w = data.iloc[:, 20].values
  
print(w)

xc_step1[index], yc_step1[index] = beam_coords(index,result.x)

print(xc_step1)
print(yc_step1)

In [ ]:
xsym, ysym = sym_to_beam((x,y),-0)

In [ ]:
print(selected_data)

In [ ]:
print(xc_step1)
print(yc_step1)

In [ ]:
print(xsym)
print(ysym)

In [ ]:
square_cut = np.logical_and.reduce([
        np.abs(xsym - xc_step1[index]) < 50.,
        np.abs(ysym - yc_step1[index]) < 50.])

# old code
## square_cut = np.logical_and.reduce([
#         np.abs(xsym - xc_step1[index]) < 6.,
#         np.abs(ysym - yc_step1[index]) < 6.])

In [ ]:
print(square_cut)

In [ ]:
print(square_cut)

In [ ]:
print(data_reset['MM1_all_charge'].unique())

In [ ]:
print(selected_data)

In [ ]:
print(mm_basic_cut[0])

In [ ]:
 #square_cut = np.tile(square_cut, len(mm_basic_cut[0]) // len(square_cut) + 1)[:len(mm_basic_cut[0])]

In [ ]:
print(square_cut)

In [ ]:
# Reset index of the DataFrame before boolean indexing
#data_reset = data.reset_index(drop=True)

#mm_basic_cut[0] = np.full(len(data_reset), False)  # Initialize with the length of data_reset

for alg_steps in range(20): #if in 20 steps not converged.. terminates
    #1st step of the algorithm
    #h2 =[None] * 4
    #projX =[None] * 4
    #projY =[None] * 4

    h2 =[None]
    projX =[None]
    projY =[None]

    #for index in range(4):
    #for index in range(1):
    index = 0 
    pad = index +1      

    h1 = ROOT.TH2F('',"", 28,xc_step1[index]-7.,xc_step1[index]+7., 28, yc_step1[index]-4.,yc_step1[index]+4.)
    h2[index] = ROOT.TH2F('',f"PAD0 distribution in Local frame",28,xc_step1[index]-4.,xc_step1[index]+4.,28,yc_step1[index]-7.,yc_step1[index]+7.)

    h2[index].GetXaxis().SetTitle(f'x symmetry axis')
    h2[index].GetYaxis().SetTitle(f'y symmetry axis')
        
    #update estimations about centers in beam frame
    xc_step1[index], yc_step1[index] = beam_coords(index,result.x)

    xy_values = data.iloc[:, -4].values
    x_indices = range(0, len(xy_values), 3)
    y_indices = range(1, len(xy_values), 3)
    #xsym, ysym = sym_to_beam((x,y),-ui)
    xsym, ysym = sym_to_beam((x,y),-ui)  
    
    
    # xsym, ysym = sym_to_beam((x,y),-ui)
    #xsym = xsym[mm_basic_cut[index] & square_cut]
    #ysym = ysym[mm_basic_cut[index] & square_cut]
    
    #square_cut = np.logical_and(np.abs(xsym-xc_step1[index])<50., np.abs(ysym-yc_step1[index])<50.)
    square_cut = np.logical_and(np.abs(xsym-xc_step1[index])<50., np.abs(ysym-yc_step1[index])<50.)
    
    n = len(data[mm_basic_cut[index]*square_cut])
   
    selected_indices = np.where(mm_basic_cut[index])[0]
    xsym = xsym[selected_indices]
    ysym = ysym[selected_indices]
        
       
    xsym_selected = xsym[selected_indices]
    ysym_selected = ysym[selected_indices]
        
    w = np.ones(len(xsym_selected))
    weights_selected = w[selected_indices]

        
    for i in range(len(xsym_selected)):
        h1.Fill(xsym_selected[i], ysym_selected[i], weights_selected[i])
        
    #w = np.ones(n)
    #h1.FillN(n,xsym,ysym,w)
        
    #w = data.iloc[:,7+index*4][mm_basic_cut[index]*square_cut].values
    #print(len(np.array(mm_basic_cut[0])))
    #print(len(np.array(square_cut)))

    w = selected_data.iloc[:, 20].values
    #2[index].FillN(n,xsym,ysym,w)
    for i in range(len(xsym_selected)):
        h2[index].Fill(xsym_selected[i], ysym_selected[i],w[i])
        
    h2[index].Divide(h1)
        #clear out
    
    for i in range(h2[index].GetNbinsX()+1):
        for j in range(h2[index].GetNbinsY()+1):
            if h2[index].GetBinContent(i,j)<2.:
                h2[index].SetBinContent(i,j,0)

        # projections to symmetry axis
    projX[index] = h2[index].ProjectionX(f'PAD0_px ')
    projX[index].SetTitle(f'PAD0 x projection (Local Frame)')
    projX[index].GetXaxis().SetTitle(f'x symmetry axis')

    # Check if the integral is non-zero before scaling
    integral_projX = projX[index].Integral()
    if integral_projX != 0:
        projX[index].Scale(1. / integral_projX)

    projY[index] = h2[index].ProjectionY(f'PAD0_py ')
    projY[index].SetTitle(f'PAD0 y projection (Local Frame)')
    projY[index].GetXaxis().SetTitle(f'y symmetry axis')

    # Check if the integral is non-zero before scaling
    integral_projY = projY[index].Integral()
    if integral_projY != 0:
        projY[index].Scale(1. / integral_projY)
        
        
        #projections to symmetry axis
        #projX[index] = h2[index].ProjectionX(f'PAD{pad}_px ')
        #projX[index].SetTitle(f'PAD0 x projection (Local Frame)')
        #projX[index].GetXaxis().SetTitle(f'x symmetry axis')
        #projX[index].Scale(1./projX[index].Integral())

        #projY[index] = h2[index].ProjectionY(f'PAD{pad}_py ')
        #projY[index].SetTitle(f'PAD0 y projection (Local Frame)')
        #projY[index].GetXaxis().SetTitle(f'y symmetry axis')
        #projY[index].Scale(1./projY[index].Integral())


    c.Clear()
    #c.Divide(2,2)
    #for i in range(4):
    for i in range(1):
        #c.cd(i+1)
        h2[i].Draw("surf3")#"lego2z"
    c.Update()
    c.Draw()
    #input('Enter to continue ')
   #for index in range(4):
    
#     for index in range(1):
    index = 0   
    pad = index +1
    #fit
    mean = projX[index].GetMean()
    projX[index].SetAxisRange(mean-10,mean+10, 'X')
    fitf = ROOT.TF1("fitf", weird_gaussian,mean-6,mean+6,4)
    #fitf = ROOT.TF1("fitf", parabola,mean-6,mean+6,3)
    
    binmax = projX[index].GetMaximumBin()
    maximum = projX[index].GetXaxis().GetBinCenter(binmax)
    par = np.array([maximum,mean-2.,mean+2.,projX[index].GetStdDev()])
    #par = np.array([-projX[index].GetStdDev(),mean,maximum])
    fitf.SetParameters(par)
        
    print("Before fitting:")
    print("projX[index] integral:", projX[index].Integral())
    print("fitf parameters:", [fitf.GetParameter(i) for i in range(fitf.GetNpar())])
        
    # Before fitting
    print(f"Before fitting - projX[{index}] information:")
    print(f"Number of entries: {projX[index].GetEntries()}")
    print(f"Integral: {projX[index].Integral()}")
    print(f"Mean: {projX[index].GetMean()}")
    print(f"RMS: {projX[index].GetRMS()}")

    # Print the histogram parameters
    print("Histogram parameters:")
    print(f"X axis range: {projX[index].GetXaxis().GetXmin()} to {projX[index].GetXaxis().GetXmax()}")
    print(f"Y axis range: {projX[index].GetYaxis().GetXmin()} to {projX[index].GetYaxis().GetXmax()}")

    # Print fit function parameters
    print("Fit function parameters:")
    for i in range(fitf.GetNpar()):
        print(f"Parameter {i}: {fitf.GetParameter(i)}")

    fit_res = projX[index].Fit(fitf,"RSQ")

    mux = (fit_res.Parameter(1)+fit_res.Parameter(2))/2
    xc_er[index] = np.sqrt(fit_res.ParError(1)**2+fit_res.ParError(1)**2) #approximately
    print(f'fit X pars errors: {xc_er[index]}')
        
        
    #same for y
    mean = projY[index].GetMean()
    projY[index].SetAxisRange(mean-10,mean+10, 'X')
    fitf = ROOT.TF1("fitf", weird_gaussian,mean-6,mean+6,4)
    #fitf = ROOT.TF1("fitf", parabola,mean-6,mean+6,3)
    
    binmax = projY[index].GetMaximumBin()
    maximum = projY[index].GetXaxis().GetBinCenter(binmax)
    par = np.array([maximum,mean-2.,mean+2.,projY[index].GetStdDev()])
    #par = np.array([-projY[index].GetStdDev(),mean,maximum])
    fitf.SetParameters(par)
    
       
    fit_res = projY[index].Fit(fitf,"RSQ")

    muy = (fit_res.Parameter(1)+fit_res.Parameter(2))/2
    yc_er[index] = np.sqrt(fit_res.ParError(1)**2+fit_res.ParError(1)**2)
    print(f'fit Y pars errors: {yc_er[index]}')

        
    #independent est. for pad centers (beam frame)
    mux = h2[index].GetMean(1)
    muy = h2[index].GetMean(2)
    xc_step1[index], yc_step1[index] = sym_to_beam((mux,muy),ui)

         
    #update data points (input in minimization) according to step1
    xdata = xc_step1
    ydata = yc_step1

    #udate sigmas
    #for index in range(4):
    #for index in range(1):

    #sigmas
    sumx = 0
    sumy = 0
    sumq = 0
        

        #for i in range(pad_hist[index].GetNbinsX()):
            #for j in range(pad_hist[index].GetNbinsY()):
                #delta_bin = pad_hist[index].GetBinError(i,j)
                #xi = pad_hist[index].GetXaxis().GetBinCenter(i)
                #yi = pad_hist[index].GetYaxis().GetBinCenter(j)
                #sumx += delta_bin**2*(xi-xdata[index])**2
                #sumy += delta_bin**2*(yi-ydata[index])**2
                #sumq += pad_hist[index].GetBinContent(i,j)
                
    for i in range(h2.GetNbinsX()):
        for j in range(h2.GetNbinsY()):
            delta_bin = h2.GetBinError(i,j)
            xi = h2.GetXaxis().GetBinCenter(i)
            yi = h2.GetYaxis().GetBinCenter(j)
            sumx += delta_bin**2*(xi-xdata[index])**2
            sumy += delta_bin**2*(yi-ydata[index])**2
            sumq += h2.GetBinContent(i,j)

    if sumq != 0:
        sigmax[index] = np.sqrt(sumx/sumq)
        sigmay[index] = np.sqrt(sumy/sumq)
    else:
        # Handle the case where sumq is zero, e.g., set sigmax and sigmay to some default values.
        #sigmax[index] = pad_hist[index].GetRMS(1)
        #sigmay[index] = pad_hist[index].GetRMS(2)
        sigmax[index] = h2.GetRMS(1)
        sigmay[index] = h2.GetRMS(2)
            
            
    sigmax[index] = np.sqrt(sumx/sumq)
    sigmay[index] = np.sqrt(sumy/sumq)
    
    # minimize chi2 to get param values
    initial_guess = [x0i,y0i,ui]

    result = minimize(chi2, initial_guess)
    if result.success:
        x0i,y0i,ui = result.x
        print(result)
    else:
        raise ValueError(result.message) 

#estimations of step2
    #for index in range(4):
    xc_step2[index], yc_step2[index] = beam_coords(index,result.x)

    #update(xc_step1,yc_step1)
    print(f'step1: x_center ={xc_step1} ')
    print(f'step1: y_center ={yc_step1} ')
    print(f'step2: x_center ={xc_step2} ')
    print(f'step2: y_center ={yc_step2} ')

    ftol = 1e-06 #default
    for i in range(len(result.x)):
        hess_inv_i = result.hess_inv[i,i]
        uncertainty_i = np.sqrt(max(1, abs(result.fun)) * ftol * hess_inv_i)
        print('p^{0} = {1:12.4e} ± {2:.1e}'.format(i, result.x[i], uncertainty_i))


    x_check = np.abs(xc_step1-xc_step2)<=np.abs(0.05) # errors put of 0.05-0.04 for converge
    y_check= np.abs(yc_step1-yc_step2)<=np.abs(0.04)
    if x_check[0]==True and y_check[0]==True:
        if np.all(x_check==x_check[0]) and np.all(y_check==y_check[0]):
            print('--------------------------------------')
            print('CONVERGENCE REACHED')
            print('--------------------------------------')
            centers = np.empty((2,4))
            centers[0]=xc_step2
            centers[1]=yc_step2
            print(centers)
            with open(f'/eos/home-a/akallits/SWAN_projects/pico/run_{run[0]}_{run[1]}_info/pad_centers.txt', 'w') as f: #save results to txt
                np.savetxt(f,centers, delimiter=" ")
            exit(0)

print('--------------------------------------')
print('TOO MANY ITERATIONS WITHOUT CONVERGING')
print('--------------------------------------')   
 

In [ ]:
print("Length of mm_basic_cut[0]:", len(mm_basic_cut[0]))
print("Length of square_cut:", len(square_cut))

print("mm_basic_cut[0]:", mm_basic_cut[0])
print("square_cut:", square_cut)

In [ ]:
w = selected_data.iloc[:, 10].values
print(w)

In [ ]:
print(len(mm_basic_cut[0]))
print(len(square_cut))

In [ ]:
print(data_reset.index)
print(mm_basic_cut[0])

In [ ]:
print(data_reset.head())  # Display the first few rows of your DataFrame

In [ ]:
#print("xsym_selected:", xsym_selected)
#print("ysym_selected:", ysym_selected)

In [ ]:
print("square_cut:", square_cut)

In [ ]:
#print(selected_indices)

In [ ]:
print(n)

In [ ]:
print(selected_data)

In [ ]:
print(mm_basic_cut)

In [ ]:
#print("weights_selected:", weights_selected)


In [ ]:
print("Selected Data:")
print(selected_data)


## Different version trial

In [ ]:
vall = uproot.concatenate(scan_files,
['Baseline_Window','MCP1_baseline_level', 'MCP1_baseline_rms', 'MCP1_global_maximum_y', \
'MCP1_global_maximum_x', 'MCP1_start_x', 'MCP1_e_peak_end_x', \
'MCP1_e_charge', 'MCP1_all_charge', 'MCP1_twentypercent_time', \
'MCP1_naive_time', 'MCP1_cfd_time', 'MCP1_sigmoid_timepoint', \
'MM1_baseline_level', 'MM1_baseline_rms', 'MM1_global_maximum_y', \
'MM1_global_maximum_x', 'MM1_start_x', 'MM1_e_peak_end_x', \
'MM1_e_charge', 'MM1_all_charge', 'MM1_twentypercent_time', \
'MM1_naive_time', 'MM1_cfd_time', 'MM1_sigmoid_timepoint', \
'MM1_sigmoid_parameters','MM1_sigmoid_chi_square',\
'MM1_sigmoid_failed','MM1_Fullsigmoid_charge',\
'MM1_Fullsigmoid_chi2', 'MM1_Fullsigmoid_fail',\
'MM1_Fullsigmoid_parameters','NumberofTracks',\
'POS0_track_hit','TrackChi2','SlopeXZ','SlopeYZ',\
 'SRSnumber'
 
# 'ntracks', 'xy', 'track_flag'
], library='np')


In [ ]:
#track_flag  = vall['NumberofTracks']
track_flag = data.track_flag

In [ ]:
vall

In [ ]:
#track_flag  = vall['NumberofTracks']
valid_tracks = track_flag == 1


In [ ]:
# mcp1_v   = vall['MCP1_global_maximum_y'][valid_tracks]
# mcp1_q   = vall['MCP1_all_charge'][valid_tracks]
# mcp1_qe  = vall['MCP1_e_charge'][valid_tracks]

# mcp1_t   = vall['MCP1_sigmoid_timepoint'][valid_tracks]
# mcp1_tnaive   = vall['MCP1_naive_time'][valid_tracks]

# mm1_v    = vall['MM1_global_maximum_y'][valid_tracks]
# mm1_t    = vall['MM1_sigmoid_timepoint'][valid_tracks]
# mm1_tnaive    = vall['MM1_naive_time'][valid_tracks]

# mm1_qe   = vall['MM1_e_charge'][valid_tracks]
# mm1_qall = vall['MM1_all_charge'][valid_tracks]

# xy       = vall['POS0_track_hit'][valid_tracks]
# chi2     = vall['TrackChi2'][valid_tracks]
# ntracks  = vall['SRSnumber'][valid_tracks]


In [ ]:
mcp1_v   = data.mcp1_gpeak[valid_tracks]
mcp1_q   = data.mcp1_qall[valid_tracks]
mcp1_qe  = data.MCP1_e_charge[valid_tracks]

mcp1_t   = data.mcp1_t[valid_tracks]
mcp1_tnaive   = data.MCP1_naive_time[valid_tracks]

mm1_v    = data.MM1_global_maximum_y[valid_tracks]
mm1_t    = data.MM1_sigmoid_timepoint[valid_tracks]
mm1_tnaive    = data.MM1_naive_time[valid_tracks]

mm1_qe   = data.MM1_e_charge[valid_tracks]
mm1_qall = data.MM1_all_charge[valid_tracks]

xy       = data.POS0_track_hit[valid_tracks]
chi2     = data.chi2[valid_tracks]
ntracks  = data.SRSnumber[valid_tracks]

In [ ]:
mcp1_v.size

In [ ]:
xy     

In [ ]:
pad_order = [0]

In [ ]:
fxs = np.array([55.1422224110843,55.19817009953217,45.12237860710425,45.178326295552125,35.1025348031242,35.15848249157207])
fys = np.array([57.34553425762093,47.32569045364089,57.28958656917306,47.269742765193016,57.23363888072518,47.213795076745136])

centers = np.stack((fxs,fys), axis=0)
cross2 = np.array([[40.1408732879686, 52.25194393206103],[50.160717093663706, 52.30789131335189]])
theta = 0.0055836

# center = np.array([45.075, 52.27945])
center = np.array([48.67, 44.68])

In [ ]:
dt1 = mm1_t - mcp1_t

In [ ]:
dt1_naive = mm1_tnaive - mcp1_tnaive

In [ ]:
NPADS = 1

In [ ]:
colors_mcp = ['black', 'maroon', 'blue', 'magenta']
colors = ['red', 'blue', 'magenta', 'green', 'maroon', 'olive', 'yellow']
pnames = ['Pad 1', 'Pad 2', 'Pad 3', 'Pad 4', 'Pad 5', 'Pad 6', 'Pad 7']

In [ ]:
N = len(mcp1_v)
print(N)

In [ ]:
fig, axs = plt.subplots(figsize=(15, 10))
#for i in range(len(mcp_vs)):
axs.hist(mcp1_v, bins=150, range=(0.0, 0.4), histtype='step', color='black', lw=2, label=f'MCP')
axs.hist(mm1_v, bins=150, range=(0.0, 0.2), histtype='step', color='red', lw=2, label=f'MM pad')
axs.set_yscale('log')
axs.set_xlabel('Amplitude [V]')
axs.set_title('')
axs.legend()
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(15, 10))
#for i in range(len(mcp_qs)):
axs.hist(mcp1_qe, bins=100, range=(0.0, 8), histtype='step', color='black', lw=2, label=f'MCP')
axs.hist(mm1_qe, bins=100, range=(0.0, 8), histtype='step', color='red', lw=2, label=f'MM pad')
axs.set_yscale('log')
axs.set_xlabel('Charge [pC]')
axs.set_title('')
axs.legend()
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(15, 10))
#for i in range(len(mm_vs)):
axs.hist(mm1_v, bins=100, range=(0.0, 0.175), histtype='step', color='red', lw=2, label=f'MM Pad')
axs.hist(mcp1_v, bins=100, range=(0.0, 0.175), histtype='step', color='black', lw=2, label=f'MCP')
# axs.set_yscale('log')
axs.set_xlabel('Amplitude [V]')
axs.set_title('MM Pad')
axs.legend()
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(15, 10))
#for i in range(len(mm_qes)):
axs.hist(mm1_qe, bins=100, range=(0.0, 10), histtype='step', color="black", lw=2, label=f'MM Pad')
axs.hist(mcp1_q, bins=100, range=(0.0, 10), histtype='step', color='red', lw=2, label=f'MCP')
axs.set_yscale('log')
axs.set_xlabel('e-Peak Charge [pC]')
axs.set_title('')
axs.legend()
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(15, 10))
#for i in range(len(mm_qalls)):
axs.hist(mm1_qall, bins=100, range=(0.0, 50), histtype='step', color='black', lw=2, label=f'MM Pad')
axs.hist(mcp1_qe, bins=100, range=(0.0, 10), histtype='step', color='red', lw=2, label=f'MCP')
axs.set_yscale('log')
axs.set_xlabel('Total Charge [pC]')
axs.set_title('MM Pads')
axs.legend()
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(15, 10))

#for i in range(len(mm_vs)):
axs.scatter(mm1_v, mm1_qe, s=0.6, color='black', label=f'MM Pad')
    
axs.set_xlabel('Amplitude [V]')
axs.set_ylabel('e-peak Charge [pC]')
axs.set_xlim(0, 0.18)
axs.set_ylim(0, 10)
axs.legend()

plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(15, 10))

#for i in range(len(mm_vs)):
axs.scatter(mm1_v, mm1_qall, s=0.6, color='black', label=f'MM Pad')
    
axs.set_xlabel('Amplitude [V]')
axs.set_ylabel('Total Charge [pC]')
axs.set_xlim(0, 0.18)
axs.set_ylim(0, 50)
axs.legend()

plt.show()

In [ ]:
mm_basic_cut = np.logical_and((mm1_v > 0.008),(mm1_v < 0.15))

In [ ]:
mm_basic_cut = (data['mm1_gpeak'] > 0.008) & (data['mm1_gpeak'] < 0.15)


# Tracks

In [ ]:
#ntracks = vall['ntracks']
#ntracks = vall['SRSnumber']
#xy = vall['POS0_track_hit']
#chi2 = vall['TrackChi2']

In [ ]:
xy[:]

In [ ]:
print(xy.shape)

In [ ]:
fig, axs = plt.subplots(figsize=(11, 11))
axs.scatter([point[0] for point in xy], [point[1] for point in xy], s=1, color='black', label='All tracks')
axs.set_xlim(0, 100)
axs.set_ylim(0, 100)
axs.legend(loc=3, markerscale=5)
axs.set_xlabel('X [mm]')
axs.set_ylabel('Y [mm]')
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(15, 10))
axs.hist(chi2, bins=100, range=(0, 50), histtype='step', color='black', lw=2)

axs.set_yscale('log')
axs.set_xlabel(r'Track $\chi^2$')
plt.show()

In [ ]:
# Define conditions for filtering
vv_thr = 0.005
qq_thr = 3.

mm_basic_cut = (data['mm1_gpeak'] > 0.008) & (data['mm1_gpeak'] < 0.15)
condition_xyc1 = mm_basic_cut & (data['mm1_gpeak'] > vv_thr) & (data['mm1_qall'] < 50)
condition_xyq1 = mm_basic_cut & (data['mm1_qall'] > qq_thr)

# Apply conditions to the DataFrame
xyc1 = data[condition_xyc1]
xyq1 = data[condition_xyq1]


In [ ]:
print(xy.shape)
print(xyc1)

In [ ]:
#old code 
# filtered_xy = xy[xyc1]

# print("Shape of filtered_xy:", filtered_xy.shape)

# print("Contents of filtered_xy:", filtered_xy)


In [ ]:
filtered_indices = xyc1.index[xyc1.all(axis=0)]  # Extract indices where all conditions are True
filtered_xy = xy[filtered_indices]  # Filter xy using the extracted indices

print("Shape of filtered_xy:", filtered_xy.shape)
print("Contents of filtered_xy:", filtered_xy)


In [ ]:
#xyc1 = np.where(np.logical_and(mm_basic_cut, mm1_v > vv_thr)*(mm1_qall < 50))
condition_xyc1 = mm_basic_cut & (data['mm1_gpeak'] > vv_thr) & (data['mm1_qall'] < 50)

print("xyc1 =", condition_xyc1)

In [ ]:
filtered_xy = xy[xyc1]  # Filter the data based on the boolean mask xyc1

fig, axs = plt.subplots(figsize=(11, 11))

axs.scatter([point[0] for point in xy], [point[1] for point in xy], s=1, color='black', label='All tracks')

axs.scatter([point[0] for point in filtered_xy], [point[1] for point in filtered_xy], s=0.6, color=colors[0], label=f'{pnames[0]} Vpeak > {vv_thr}V')

axs.set_xlim(20, 80)

axs.set_ylim(20, 80)

axs.legend(loc=1, markerscale=10)

axs.set_xlabel('X [mm]')

axs.set_ylabel('Y [mm]')

plt.show()

In [ ]:
filtered_xy = xy[xyq1]  # Filter the data based on the boolean mask xyc1

fig, axs = plt.subplots(figsize=(11, 11))

axs.scatter([point[0] for point in xy], [point[1] for point in xy], s=1, color='black', label='All tracks')

axs.scatter([point[0] for point in filtered_xy], [point[1] for point in filtered_xy], s=0.6, color=colors[0], label=f'{pnames[0]} Qall > {qq_thr}pC')

axs.set_xlim(20, 80)

axs.set_ylim(20, 80)

axs.legend(loc=1, markerscale=10)

axs.set_xlabel('X [mm]')

axs.set_ylabel('Y [mm]')

plt.show()

In [ ]:
heatmap, yedges, xedges = np.histogram2d([point[1] for point in xy], [point[0] for point in xy], range=[[20,80],[20,80]], bins=(201, 201))

fig, axs = plt.subplots(figsize=(12, 10))
axs.set_xlabel('X [mm]')
axs.set_ylabel('Y [mm]')
c = axs.pcolormesh(xedges, yedges, heatmap, cmap='hot_r', rasterized=True)
fig.colorbar(c, ax=axs)
plt.show()

In [ ]:
%jsroot off
c = ROOT.TCanvas('c1', '', 1000, 1000)

hxy = ROOT.TH2D("hxy_all", '', 200, 20, 80, 200, 20, 80)
hxy.FillN(len([point[0] for point in xy]), np.array([point[0] for point in xy]), np.array([point[1] for point in xy]), 0, 1)
# hxy.FillN(len(xy[:,0]), xy[:, 0], xy[:, 1], 0, 1)

for i in range(201*201):
    if hxy.GetBinContent(i) < 4: #Tracks per bin 
        hxy.SetBinContent(i, 0)
hxy.Draw('colz')
c.Draw()

In [ ]:
%jsroot off
c = ROOT.TCanvas('c1', '', 2000, 1000)
c.Divide(2, 1)

hxy_mcp = ROOT.TH2D("hxy_mcp", '', 200, 20, 80, 200, 20, 80)
hxy_mcp.FillN(mcp1_q.size, np.array([point[0] for point in xy]), np.array([point[1] for point in xy]), mcp1_q, 1)


for i in range(201*201):
    if hxy.GetBinContent(i) < 4 or hxy_mcp.GetBinContent(i) < 0:
        hxy_mcp.SetBinContent(i, 0)
    else:
        hxy_mcp.SetBinContent(i, hxy_mcp.GetBinContent(i)/hxy.GetBinContent(i))


c.cd(1)
hxy_mcp.Draw('colz')
c.cd(2)
hxy_mcp.Draw('lego2z')
c.Draw()

In [ ]:
%jsroot off
c = ROOT.TCanvas('c1', '', 2000, 1000)
c.Divide(2, 1)

hxy_mm1 = ROOT.TH2D("hxy_mm1", '', 200, 20, 80, 200, 20, 80)
hxy_mm1.FillN(mm1_qall.size, np.array([point[0] for point in xy]), np.array([point[1] for point in xy]), mm1_qall, 1)


for i in range(201*201):
    if hxy.GetBinContent(i) < 4 or hxy_mm1.GetBinContent(i) < 0:
        hxy_mm1.SetBinContent(i, 0)
    else:
        hxy_mm1.SetBinContent(i, hxy_mm1.GetBinContent(i)/hxy.GetBinContent(i))


c.cd(1)
hxy_mm1.Draw('colz')
c.cd(2)
hxy_mm1.Draw('lego2z')
c.Draw()

In [ ]:
xyc1

In [ ]:
fxs2 = np.zeros(6)
fys2 = np.zeros(6)

In [ ]:
fxs2

In [ ]:
filtered_xy # filtered_xy = xy[xyq1] tracks on this combined cut over charge 

In [ ]:
xlims = [[45,65], [40,70], [30,60], [30,60], [20,50], [20,50]]
ylims = [[45,65], [30,60], [45,65], [30,60], [40,70], [30,60]]

xtemps = [[point[0] for point in filtered_xy]]
ytemps = [[point[1] for point in filtered_xy]]
qtemps = [mm1_qall[xyc1]]

In [ ]:
xtemps

In [ ]:
#mm_basic_cut=[]
mcp_overflow = (mcp1_v > 0) * (mcp1_v <= 0.51)
chi2_cut = (chi2 <= 50)
mcp_basic_cut = mcp_overflow * chi2_cut

for i in range(NPADS):
    mm_overflow = (mm1_v[i] > 0.) * (mm1_v[i] < 0.15) * (mm1_qall[i] > 0) * (mm1_qall[i] < 30)
    qall_line = (mm1_qall[i] > 1. * mm1_v[i])

    # Make sure mcp_basic_cut has the same size as filtered_xy
    cut = np.logical_and(mm_overflow * mcp_basic_cut[:len(filtered_xy)], qall_line)
    #cut = np.logical_and(mm_overflow * mcp_basic_cut, qall_line)
    mm_basic_cut = mm_basic_cut[:len(cut)] * cut

In [ ]:
xy0_data = [point[0] for point in filtered_xy]
xy1_data = [point[1] for point in filtered_xy]

# Convert the lists to NumPy arrays
xy0_data = np.array(xy0_data)
xy1_data = np.array(xy1_data)

# Create a boolean mask for filtering
mm_basic_mask = np.array(mm_basic_cut).all(axis=0)

# Use the mask to filter xy0_data and xy1_data
xy0_filtered_mcp = xy0_data[mm_basic_mask]
xy1_filtered_mcp = xy1_data[mm_basic_mask]

In [ ]:
# Create boolean masks
mask_mcp_basic = mcp_basic_cut[:len(filtered_xy)]

# Filter the data using the boolean masks
filtered_x = np.array(xy0_data)
filtered_y = np.array(xy1_data)
filtered_x = filtered_x[mask_mcp_basic]
filtered_y = filtered_y[mask_mcp_basic]

# Now create the histogram
hist_tracks, yedges, xedges = np.histogram2d(filtered_y, filtered_x, range=[[20, 80], [20, 80]], bins=200)

# Apply threshold to the histogram
hist_tracks = np.where(hist_tracks < 4, 0, hist_tracks)

# Create the plot
fig, axs = plt.subplots(figsize=(12, 10))
axs.set_xlabel('X [mm]')
axs.set_ylabel('Y [mm]')
c = axs.pcolormesh(xedges, yedges, hist_tracks, cmap='hot_r', rasterized=True)
fig.colorbar(c, ax=axs)
plt.show()

In [ ]:
c = ROOT.TCanvas('cc', '', 1000, 800)
xytrks = ROOT.TH2D("xytrks",'All tracks multiplicity',200,20,80,200,20,80)

xytrks.GetXaxis().SetTitle("X [mm]")
xytrks.GetYaxis().SetTitle("Y [mm]")

n = len(filtered_y)
xytrks.FillN(n,filtered_x,filtered_y, 0, 1)
xytrks.Draw('colz')
c.Draw()

In [ ]:
#root canvas for display
c = ROOT.TCanvas()
ROOT.gStyle.SetOptFit(1111)
#tracks multiplicity
xytrks = ROOT.TH2F("xytrks",f' All tracks multiplicity',100,20,70,100,20,70)

xytrks.GetXaxis().SetTitle("x [mm]")
xytrks.GetYaxis().SetTitle("y [mm]")

n = len(filtered_y)
x = filtered_x
y = filtered_y
w = np.ones(n) # weights (w) to ones for all data points, you can create an array of ones with the same length as your data and assign it to w like this:
xytrks.FillN(n,x,y,w)
#clear out areas with low number of trks (not enough statistics)
for i in range(xytrks.GetNbinsX()+1):
    for j in range(xytrks.GetNbinsY()+1):
        if xytrks.GetBinContent(i,j)<2:
            xytrks.SetBinContent(i,j,0)

xytrks.SetTitle(f'xy tracks with multiplicity >2')
xytrks.Draw("colz")
c.Draw()

In [ ]:
filtered_x

In [ ]:
# we find the pads here!
xytrksW_mm = [] #weighted histos
pad_hist = []

In [ ]:
%jsroot off
c = ROOT.TCanvas('c1', '', 800, 800)
#xytrks = ROOT.TH2D("xytrks",'All tracks multiplicity',200,20,80,200,20,80)
h1 = ROOT.TH2F(f'xytrksWeighted',f' xy tracks weighted by PAD  e-peak row charge',100,20,70,100,20,70)
for index in range(1):
    pad = index +1
    
    h1.GetXaxis().SetTitle("x [mm]")
    h1.GetYaxis().SetTitle("y [mm]")

    n = len(filtered_xy)
    x = filtered_x[mm_basic_cut[index]]
    y = filtered_y[mm_basic_cut[index]]
    #w = mm1_qe[mm_basic_cut[index]]
    w = np.ones(n)
    h1.FillN(n,x,y,w)
    
    h1.Divide(xytrks)
    
    xytrksW_mm.append(h1)
    h2 = h1.Clone()
    h2.SetName(f'pad_hist')
    h2.SetTitle(f' PAD')
    for i in range(h2.GetNbinsX()+1):
        for j in range(h2.GetNbinsY()+1):
            if h2.GetBinContent(i,j)<2.:
                h2.SetBinContent(i,j,0)
    
    pad_hist.append(h2)
    
# Create the plot
for i in range(1):
    c.cd(i+1)
    xytrksW_mm[i].Draw("lego2z")#"lego2z/colz"
c.Draw()

In [ ]:
x

In [ ]:
#distance_xy = 9.82/2 in mm half of the distance of the pad to select tracks passing from this area 
distance_xy = 0

centers_s = sym_to_beam((centers[0,:] - center[0],centers[1,:] - center[1]), theta)
x_s, y_s = sym_to_beam([(point[0] - center[0], point[1] - center[1]) for point in xy], theta)

#x_s, y_s = sym_to_beam([point[0] - center[0] for point in xy], [point[1] - center[1] for point in xy], theta)
#x_s, y_s = sym_to_beam([(point[0] - center[0], point[1] - center[1]) for point in xy], theta)

#cross2_s = np.array(sym_to_beam((cross2[:,0] - center[0],cross2[:,1] -center[1]), theta)).T

In [ ]:
centers_s[0]

In [ ]:
centers_s

In [ ]:
center

In [ ]:
centers[0,:] #x values 

In [ ]:
centers[1,:] #y values 

In [ ]:
#def transfo(x,y,x0,y0, theta):
    #dx=x-x0
    #dy=y-y0
    #return (dx*np.cos(theta)+dy*np.sin(theta), -dx*np.sin(theta)+dy*np.cos(theta))
#xy_pad_cuts = []
for i in range(NPADS):
    temp_cut = np.logical_and(np.logical_and(np.abs(x_s-centers_s[0][i]) < distance_xy, np.abs(y_s-centers_s[1][i]) < distance_xy), mm_cut[i])
    #temp_cut = np.logical_and(np.logical_and(np.abs(x_s-centers_s[0][i]), np.abs(y_s-centers_s[1][i])), mm_cut[i])
    #temp_cut = mm_cut[i]    
    xy_pad_cuts.append(temp_cut)
# a1,a2 = transfo(xy[:,0],xy[:,1],center[0],center[1],theta)

# square1 = np.logical_and(np.abs(a1-centers[0,i]) < distance_xy, np.abs(xy[:, 1]-centers[1,i]) < distance_xy)

In [ ]:
fig, axs = plt.subplots(figsize=(15, 15))

axs.set_title('Local Frame')
axs.set_xlabel('X\' [mm]')
axs.set_ylabel('Y\' [mm]')

axs.axhline(0, lw=1, color='black')
axs.axvline(0, lw=1, color='black')

for i in range(NPADS):
    axs.scatter(x_s[xy_pad_cuts[i]], y_s[xy_pad_cuts[i]], s=0.8, color=colors[i], label=f'{pnames[i]}')

for i in range(NPADS):
    axs.scatter(centers_s[0][i], centers_s[1][i], color='black', s=50)
#axs.scatter(0, 0, color='black', s=100, marker='+')
#axs.scatter(center[0], center[1], color='black', s=50, marker='+')
#axs.scatter(cross2_s[0,0], cross2_s[0,1], color='black', s=50, marker='^', label='Cross')
#axs.scatter(cross2_s[1,0], cross2_s[1,1], color='black', s=50, marker='^')

#box2 = np.array([[centers_s[0][0],centers_s[1][0]]])
#box1 = np.array([[centers_s[0][2],centers_s[1][2]]])

# axs.plot(box1[:,0], box1[:,1], '--o', lw=5, ms=10, color='black')
# axs.plot(box2[:,0], box2[:,1], '-o', lw=5, ms=10, color='black')
# axs.set_xlim(-15, 15)
# axs.set_ylim(-10, 10)

leg = axs.legend(loc=2, markerscale=10)

#leg.legendHandles[6].set_sizes([])

plt.show()

In [ ]:
tranges = [
    [-8000,-7300],
    [-12700, -12000],
    [-12700, -12000],
    [-8200,-7000],
    [-2700, 1000],
    [-12700, 10000],
    [-9000,-8200],
]

In [ ]:
ROOT.gStyle.SetOptFit(1111)
c = ROOT.TCanvas("cc", '', 1200, 1800)
#c.Divide(2, 4)

ht1 = ROOT.TH1D('ht1', f'{pnames[0]};T [ps]', 200, -11500,-11000)
ht1.FillN(np.count_nonzero(mm1_cut), 1e3*dt1[mm1_cut], 0, 1)

ht1.Draw()
ht1.Fit('gaus')

c.Draw()

In [ ]:
def gaus2(x, pars):
    a = pars[0]
    mu = pars[1]
    sigma1 = pars[2]
    b = pars[3]
    sigma2 = pars[4]
    return a*ROOT.TMath.Gaus(x[0], mu, sigma1) + b*ROOT.TMath.Gaus(x[0],mu,sigma2)

In [ ]:
c1_order = np.array([0], dtype=np.int32)
print([pnames[i] for i in c1_order])

In [ ]:
distance_xy = 0
xy_pad1c = np.logical_and(np.logical_and(np.abs(x_s-centers_s[0][0]) < distance_xy, np.abs(y_s-centers_s[1][0]) < distance_xy), mm_cut[0])
xyc1_inpad1 = xy_pad1c
xy_pad5c = np.logical_and(np.logical_and(np.abs(x_s-centers_s[0][0]) < distance_xy, np.abs(y_s-centers_s[1][0]) < distance_xy), mm_cut[0])
xyc1_inpad5 = xy_pad5c


In [ ]:
qlow = 0.5
qup = 50

#nppe = 500
nppe = 500


vl5in5c1, vu5in5c1, mdt5in5c1, sdt5in5c1, emdt5in5c1, esdt5in5c1 = fff(mm1_qall[xyc1_inpad5], dt1[xyc1_inpad5], qlow, qup, nppe, False, trange=[tranges[4][0], tranges[4][1]])


In [ ]:
xxs55c1 = (vl5in5c1+vu5in5c1)*0.5
exs55c1 = (vl5in5c1-vu5in5c1)*0.5

In [ ]:
show_ranges = [
    [-14000, -12000],
    [-16500, -14000],
    [-13000, -12000],
    [-16400, -14000],
]

In [ ]:
pars5in5c1 = sat_fit(xxs55c1, mdt5in5c1, exs55c1, emdt5in5c1, iistart=0, iistop=None, xl=0.5, xu=80, pars=[-0.1, 1.0, -1, 0.5, 13500.0])

In [ ]:
#same
fig, axs = plt.subplots(2, 2, figsize=(22, 12))
iistart, iistop = 0, None
axs = axs.ravel()

a = np.linspace(5, 80, 300)


axs[0].errorbar(xxs55c1[iistart:iistop], mdt5in5c1[iistart:iistop], xerr=exs55c1[iistart:iistop], yerr=emdt5in5c1[iistart:iistop], fmt='o', ms=7, color='black', label=f'{pnames[4]}, hit {pnames[4]}')
axs[0].plot(a, expo2_np(a, pars5in5c1), color=pcolors[0])

for i in range(4):
    axs[i].grid()
    axs[i].set_xlim(None, 80)
#     axs[i].set_ylim(0, 1000)
#     axs[i].set_ylim(show_ranges[i][0], show_ranges[i][1])
    axs[i].set_xlabel('Total Charge [pC]')
    axs[i].set_ylabel('SAT [ps]')
    axs[i].legend(loc=1)

# axs[0].set_xlim(0, 5)
plt.show()

In [ ]:
# vv_thr = 0.02

# xyc1 = np.where(np.logical_and(mm1_cut, mm1_v > vv_thr)*(mm1_qall < 200))


# qq_thr = 5

# xyq1 = np.where(np.logical_and(mm1_cut, mm1_qall > qq_thr))

qq_thr = 7

# xyq1 = np.where(np.logical_and(mm1_cut, mm1_qall > qq_thr))
# xyq2 = np.where(np.logical_and(mm2_cut, mm2_qall > qq_thr))
# xyq3 = np.where(np.logical_and(mm3_cut, mm3_qall > qq_thr))
# xyq4 = np.where(np.logical_and(mm4_cut, mm4_qall > qq_thr))
# xyq5 = np.where(np.logical_and(mm5_cut, mm5_qall > qq_thr))
# xyq6 = np.where(np.logical_and(mm6_cut, mm6_qall > qq_thr))
# xyq7 = np.where(np.logical_and(mm7_cut, mm7_qall > qq_thr))

xyq1 = np.where(np.logical_and(cc1, mm1_qall > qq_thr))

In [ ]:
#ROOT.gStyle.SetOptFit(1111)
#c = ROOT.TCanvas("cc", '', 1200, 1800)
#c.Divide(2, 4)

#ht1 = ROOT.TH1D('ht1', f'{pnames[0]};T [ps]', 100, -8000,-7300)
#ht1.FillN(np.count_nonzero(xyq1), 1e3*dt1[xyq1], 0, 1)

#ht2 = ROOT.TH1D('ht2', f'{pnames[1]};T [ps]', 100, -12700, -12000)
#ht2.FillN(np.count_nonzero(xyq2), 1e3*dt2[xyq2], 0, 1)

#ht3 = ROOT.TH1D('ht3', f'{pnames[2]};T [ps]', 100, -12700, -12000)
#ht3.FillN(np.count_nonzero(xyq3), 1e3*dt3[xyq3], 0, 1)

#ht4 = ROOT.TH1D('ht4', f'{pnames[3]};T [ps]', 100, -8200,-7000)
#ht4.FillN(np.count_nonzero(xyq4), 1e3*dt4[xyq4], 0, 1)

#ht5 = ROOT.TH1D('ht5', f'{pnames[4]};T [ps]', 100, -1500, -700)
#ht5.FillN(np.count_nonzero(xyq5), 1e3*dt5[xyq5], 0, 1)...

#ht6 = ROOT.TH1D('ht6', f'{pnames[5]};T [ps]', 100,-1300, -800)
#ht6.FillN(np.count_nonzero(xyq6), 1e3*dt6[xyq6], 0, 1)

#ht7 = ROOT.TH1D('ht7', f'{pnames[6]};T [ps]', 100,-9000,-8200)
#ht7.FillN(np.count_nonzero(xyq7), 1e3*dt7[xyq7], 0, 1)

#c.cd(1)
#ht1.Draw()
#ht1.Fit('gaus')

#c.cd(2)
#ht2.Draw()
#ht2.Fit('gaus')

#c.cd(3)
#ht3.Draw()
#ht3.Fit('gaus')

#c.cd(4)
#ht4.Draw()
#ht4.Fit('gaus')

#c.cd(5)
#ht5.Draw()
#ht5.Fit('gaus')

#c.cd(6)
#ht6.Draw()
#ht6.Fit('gaus')

#c.cd(7)
#ht7.Draw()
#ht7.Fit('gaus')

#c.Draw()

In [ ]:
fig, axs = plt.subplots(figsize=(29, 15))
axs.hist(ntracks, bins=10, range=(0.0, 15), histtype='step', color='black', lw=2)

axs.set_xlabel('N tracks')
axs.set_title('Number of tracks from tracker')
# axs.legend()
plt.show()

In [ ]:
fig, axs = plt.subplots(2,2,figsize=(11, 11))
axs = axs.ravel()
for i in range(1,7):
    axs[i].scatter(xy[:,i,:][:, 0], xy[:,i,:][:, 1], s=2, color='black', label=f'All tracks z={xy[:,i][:, 2][0]}')
    axs[i].set_xlim(0, 100)
    axs[i].set_ylim(0, 100)
    axs[i].legend(loc=3, markerscale=5)
    axs[i].set_xlabel('X [mm]')
    axs[i].set_ylabel('Y [mm]')
plt.show()

In [ ]:
xy

In [ ]:
ntracks[0:10]

In [ ]:
# track_files = {'/eos/home-i/iangelis/RD51/code/ebeam/analysis/July2023/Run320/anaRun320.root':'tracks'}

In [ ]:
#file_tracker = uproot.open('/eos/home-i/iangelis/RD51/code/ebeam/analysis/July2023/Run320/anaRun320.root')

In [ ]:
file_tracker = uproot.open('/eos/project-p/picosec/testbeam/2023_July_h4/tracker/reconstructed/anaRun460.root')

In [ ]:
srs_tracks = file_tracker["tracks"]["srstriggerctr"].array(library="np")
ntracks = file_tracker["tracks"]["ntracks"].array(library="np")
ndetsintrack = file_tracker["tracks"]["ndetsintrack"].array(library="np")
distnextcluster = file_tracker["tracks"]["distnextcluster"].array()
trackchi2 = file_tracker["tracks"]["trackchi2"].array(library="np")

# totchanextcluster = ftracks['tracks']['totchanextcluster'].array(library='np')
N = np.size(srs_tracks)
distnextcluster = np.asarray(distnextcluster)
distnextcluster = np.reshape(distnextcluster, (N, 6))
# print(np.shape(distnextcluster))

track_cut = np.logical_and(
    np.logical_and(ntracks == 1, ndetsintrack == 6),
    np.all(distnextcluster >= 600, axis=1),
)
flag_vec = np.empty(N)
flag_vec = np.where(track_cut == True, 1, -111)

srst_un, srst_id = np.unique(srs_tracks, return_index=True)

# all_srs = np.intersect1d(common_srs, srst_un)
# for i in range(N_POOLS):
#     masks[i] = np.isin(uniq_srs[i], all_srs)

# mask_tracks = np.isin(srst_un, all_srs)


In [ ]:
hits = file_tracker["tracks"]["hits"].array()
# hits = np.asarray(hits)[:, 3, :]
hits = np.asarray(hits)

In [ ]:
fig, axs = plt.subplots(2,2,figsize=(11, 11))
axs = axs.ravel()
for i in range(1,4):
    axs[i].scatter(hits[:,i,:][:, 0], hits[:,i,:][:, 1], s=2, color='black', label=f'All tracks z={hits[:,i][:, 2][0]}')
    axs[i].set_xlim(0, 100)
    axs[i].set_ylim(0, 100)
    axs[i].legend(loc=3, markerscale=5)
    axs[i].set_xlabel('X [mm]')
    axs[i].set_ylabel('Y [mm]')
plt.show()

In [ ]:
hits[:,:,:][:,1]

In [ ]:
hits[:,:,:][:,13]

In [ ]:
hits[:,:,:]

In [ ]:
#define the cuts
mm_basic_cut = []

mcp_overflow = (mcp1_v>0)*(mcp1_v<=0.74)
mcp_basic_cut = mcp_overflow

for i in range(NPADS):       
    mm_overflow = (mm_vs[i] > 0.)*(mm_vs[i] < 0.36)*(mm_qalls[i] > 0)*(mm_qalls[i] < 250)
    qall_line = (mm_qalls[i] > 30. * mm_vs[i])
    cut = np.logical_and(mm_overflow * mcp_basic_cut, qall_line)
    mm_basic_cut.append(cut)


In [ ]:
# cmap = plt.get_cmap('viridis')
cmap = plt.get_cmap('plasma')
pcolors = [cmap(i) for i in np.linspace(0, 1, 5)]

In [ ]:
vv_thr = 0.01

xyc1 = np.where(np.logical_and(mm1_cut, mm1_v > vv_thr)*(mm1_qall < 250))

qq_thr = 5

xyq1 = np.where(np.logical_and(mm1_cut, mm1_qall > qq_thr))


In [ ]:
fig, axs = plt.subplots(figsize=(11, 11))
axs.scatter(hits[:, 11], hits[:, 0], s=2, color='black', label='All tracks')
axs.set_xlim(0, 100)
axs.set_ylim(0, 100)
axs.legend(loc=3, markerscale=5)
axs.set_xlabel('X [mm]')
axs.set_ylabel('Y [mm]')
plt.show()

In [ ]:
fig, axs = plt.subplots(figsize=(15, 15))
axs.scatter(hits[:, 0], hits[:, 1], s=1, color='black', label='All tracks')
axs.scatter(hits[:, 0][xyc1], hits[:, 10][xyc1], s=0.6, color=colors[0], label=f'{pnames[0]} Vpeak > {vv_thr}V')
axs.set_xlim(20, 80)
axs.set_ylim(20, 80)
axs.legend(loc=1, markerscale=10)
# axs.set_title(f"Run{rr}")
axs.set_xlabel('X [mm]')
axs.set_ylabel('Y [mm]')
plt.show()

In [ ]:
%jsroot off
c = ROOT.TCanvas('c1', '', 1000, 1000)

hxy = ROOT.TH2D("hxy_all", '', 200, 20, 80, 200, 20, 80)
hxy.FillN(xy[:,0].size, np.array(xy[:, 0]), np.array(xy[:, 1]), 0, 1)
# hxy.FillN(len(xy[:,0]), xy[:, 0], xy[:, 1], 0, 1)

for i in range(201*201):
    if hxy.GetBinContent(i) < 15:
        hxy.SetBinContent(i, 0)

hxy.Draw('colz')
c.Draw()

In [ ]:
%jsroot off
c = ROOT.TCanvas('c1', '', 2000, 1000)
c.Divide(2, 1)

hxy_mcp = ROOT.TH2D("hxy_mcp", '', 200, 20, 80, 200, 20, 80)
hxy_mcp.FillN(mcp1_q.size, np.array(xy[:, 0]), np.array(xy[:, 1]), mcp1_q, 1)


for i in range(201*201):
    if hxy.GetBinContent(i) < 15 or hxy_mcp.GetBinContent(i) < 0:
        hxy_mcp.SetBinContent(i, 0)
    else:
        hxy_mcp.SetBinContent(i, hxy_mcp.GetBinContent(i)/hxy.GetBinContent(i))


c.cd(1)
hxy_mcp.Draw('colz')
c.cd(2)
hxy_mcp.Draw('lego2z')
c.Draw()

In [ ]:
c = ROOT.TCanvas('cc','', 1000, 800)
c.Divide(3,2)

for i,j in enumerate(pad_order):
    c.cd(i+1)
    pad_hists[j].Draw('colz')
#     xytrksW_mm[i].Draw('colz')
c.Draw()

In [ ]:
qlow = 0.5
qup = 80
nppe = 500
vl1in1, vu1in1, mdt1in1, sdt1in1, emdt1in1, esdt1in1 = fff(mm1_qall[xyc_inpad1], dt1[xyc_inpad1], qlow, qup, nppe, False, trange=[tranges[0][0], tranges[0][1]])

xxs11 = (vl1in1+vu1in1)*0.5
exs11 = (vl1in1-vu1in1)*0.5


In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(22, 12))
iistart, iistop = 0, -1
axs = axs.ravel()
axs[0].errorbar(xxs23[iistart:iistop], mdt2in3[iistart:iistop], xerr=exs23[iistart:iistop], yerr=emdt2in3[iistart:iistop], fmt='o', ms=7, color=pcolors[0], label=f'{pnames[1]}, hit {pnames[2]}')
for i in range(4):
    axs[i].grid()
    axs[i].set_xlim(None, 50)
    axs[i].set_ylim(0, 1000)
    axs[i].set_ylim(show_ranges[i][0], show_ranges[i][1])
    axs[i].set_xlabel('Total Charge [pC]')
    axs[i].set_ylabel('SAT [ps]')
    axs[i].legend(loc=1)

axs[0].set_xlim(0, 5)
plt.show()


In [ ]:
# fig, axs = plt.subplots(2, 2, figsize=(22, 12))
# iistart, iistop = 0, -1
# axs = axs.ravel()

# axs[0].errorbar(xxs23[iistart:iistop], sdt2in3[iistart:iistop], xerr=exs23[iistart:iistop], yerr=esdt2in3[iistart:iistop], fmt='o', ms=7, color=pcolors[0], label=f'{pnames[1]}, hit {pnames[2]}')



# for i in range(4):
#     axs[i].grid()
#     axs[i].set_xlim(None, 50)
# #     axs[i].set_ylim(0, 1000)
# #     axs[i].set_ylim(show_ranges[i][0], show_ranges[i][1])
#     axs[i].set_xlabel('Total Charge [pC]')
#     axs[i].set_ylabel('Resolution [ps]')
#     axs[i].legend(loc=1)

# # axs[0].set_xlim(0, 5)
# plt.show()


# Total Charge based Active Pad

In [ ]:
# #same
# fig, axs = plt.subplots(2, 2, figsize=(22, 12))
# iistart, iistop = 0, -1
# axs = axs.ravel()
# a = np.linspace(5, 80, 300)

# axs[0].errorbar(xxsc22_qa[iistart:iistop], mdt2in2_qa[iistart:iistop], xerr=exsc22_qa[iistart:iistop], yerr=emdt2in2_qa[iistart:iistop], fmt='o', ms=7, color=colors[1], label=f'{pnames[1]}, hit {pnames[1]}')
# axs[0].plot(a, expo2_np(a, pars2in2_qa), color=colors[1])

# for i in range(4):
#     axs[i].grid()
#     axs[i].set_xlim(None, 80)
# #     axs[i].set_ylim(0, 1000)
# #     axs[i].set_ylim(show_ranges[i][0], show_ranges[i][1])
#     axs[i].set_xlabel('Total Charge [pC]')
#     axs[i].set_ylabel('SAT [ps]')
#     axs[i].legend(loc=1)

# # axs[0].set_xlim(0, 5)
# plt.show()


In [ ]:
# qlow = 0.5
# qup = 80

# nppe = 500


# _,_, mdt1in1_qa_cor, sdt1in1_qa_cor, emdt1in1_qa_cor, esdt1in1_qa_cor = fff(mm1_qall[xyc_pad1_qa], dt1[xyc_pad1_qa]-expo2_np(mm1_qall[xyc_pad1_qa], pars1in1_qa)*1e-3, qlow, qup, nppe, False, trange=[-5000,5000])

In [ ]:
# #same
# fig, axs = plt.subplots(2, 2, figsize=(22, 12))
# iistart, iistop = 0, -1
# axs = axs.ravel()

# axs[0].errorbar(xxsc22_qa[iistart:iistop], mdt2in2_qa_cor[iistart:iistop], xerr=exsc22_qa[iistart:iistop], yerr=emdt2in2_qa_cor[iistart:iistop], fmt='o', ms=7, color=colors[1], label=f'{pnames[1]}, hit {pnames[1]}')


# for i in range(4):
#     axs[i].grid()
#     axs[i].set_xlim(None, 80)
# #     axs[i].set_ylim(0, 1000)
# #     axs[i].set_ylim(show_ranges[i][0], show_ranges[i][1])
#     axs[i].set_xlabel('Total Charge [pC]')
#     axs[i].set_ylabel('SAT [ps]')
#     axs[i].legend(loc=1)

# # axs[0].set_xlim(0, 5)
# plt.show()


In [ ]:
# spars1in1_qa = sat_fit(xxsc11_qa, sdt1in1_qa, exsc11_qa, esdt1in1_qa, iistart=0, iistop=None, xl=0.5, xu=80, pars=[-0.1,3.0,-1,0.5, 40])


In [ ]:
# #same
# fig, axs = plt.subplots(2, 2, figsize=(22, 12))
# iistart, iistop = 0, -1
# axs = axs.ravel()
# a = np.linspace(5, 80, 300)

# axs[0].errorbar(xxsc22_qa[iistart:iistop], sdt2in2_qa[iistart:iistop], xerr=exsc22_qa[iistart:iistop], yerr=esdt2in2_qa[iistart:iistop], fmt='o', ms=7, color=colors[1], label=f'{pnames[1]}, hit {pnames[1]}')
# axs[0].plot(a, expo2_np(a, spars2in2_qa), color=colors[1])




# for i in range(4):
#     axs[i].grid()
#     axs[i].set_xlim(None, 80)
# #     axs[i].set_ylim(0, 1000)
# #     axs[i].set_ylim(show_ranges[i][0], show_ranges[i][1])
#     axs[i].set_xlabel('Total Charge [pC]')
#     axs[i].set_ylabel('Resolution [ps]')
#     axs[i].legend(loc=1)

# # axs[0].set_xlim(0, 5)
# plt.show()


In [ ]:
# gpull11 = (dt1[xyc_pad1_qa]-expo2_np(mm1_qall[xyc_pad1_qa], pars1in1_qa)*1e-3)/(expo2_np(mm1_qall[xyc_pad1_qa], spars1in1_qa)*1e-3)

In [ ]:
# cg = ROOT.TCanvas('cg', '', 1000, 800)
# cg.Divide(2,2)

# hgpull11 = ROOT.TH1D('hgp11', '', 100, -10,10)
# hgpull11.FillN(gpull11.size, gpull11, 0, 1)

# cg.cd(1)
# hgpull21.Draw()
# hgpull21.Fit('gaus')

# cg.Draw()

# Pad Centers

In [ ]:
# tranges_xx = [
#  [-12000, -11000],
#  [-16500, -16000],
#  [-15700, -15000],
#  [-17000, -13000],
# ]

# hmean_sat = np.zeros((4, 10, 10))
# hmean_rms = np.zeros((4, 10, 10))
# htemps1 = []
# htemps2 = []
# htemps3 = []
# htemps4 = []

# htemps = [htemps1,htemps2,htemps3,htemps4]

# for ip,jp in enumerate(pad_order):
#     xbins = np.linspace(centers_s[0][jp]-5, centers_s[0][jp]+5, 11)
#     ybins = np.linspace(centers_s[1][jp]-5, centers_s[1][jp]+5, 11)
    
#     for j in range(1, 11):
#         for i in range(1,11):
    
#             xybin_cut = np.where(np.logical_and((x_s > xbins[i-1])*(x_s < xbins[i]), (y_s > ybins[j-1])*(y_s < ybins[j])))
# #             qq_cut = mm_qalls[ip][xybin_cut]>3
#         #             qq_in_bin = mm1_qall[box_cut][xybin_cut]
#             dt_in_bin = 1e3*mm_dts[jp][xybin_cut]
#             hxtemp = ROOT.TH1D(f'htemp{jp}_{i}_{j}', '', 100, tranges_xx[jp][0], tranges_xx[jp][1])

#             #             hxtemp.FillN(qq_in_bin.size, qq_in_bin, 0)
#             hxtemp.FillN(dt_in_bin.size, dt_in_bin, 0)
#             hxtemp.Fit('gaus', 'Q')
#             rsf = hxtemp.Fit('gaus', 'SQ')
#             if rsf.Get():
#                 hmean_sat[jp,i-1,j-1] = rsf.Parameter(1)
#                 hmean_rms[jp,i-1,j-1] = rsf.Parameter(2) 
            
#             htemps[jp].append(hxtemp)
# #     break

In [ ]:
# fig, axs = plt.subplots(10, 10, figsize=(100, 100))
# for ip,jp in enumerate(pad_order):
#     xbins = np.linspace(centers_s[0][jp]-5, centers_s[0][jp]+5, 11)
#     ybins = np.linspace(centers_s[1][jp]-5, centers_s[1][jp]+5, 11)
#     for j in range(1, 11):
#         for i in range(1,11):
    
#             xybin_cut = np.where(np.logical_and((x_s > xbins[i-1])*(x_s < xbins[i]), (y_s > ybins[j-1])*(y_s < ybins[j])))
#             axs[j-1,i-1].scatter(x_s[xybin_cut], y_s[xybin_cut])
#             axs[j-1,i-1].set_xlim(-11,11)
#             axs[j-1,i-1].set_ylim(-11,11)
#     break
# plt.show()

In [ ]:
# %jsroot off
# c = ROOT.TCanvas('cbins','', 8000, 8000)
# c.Divide(10, 10)
# for i in range(0, 100):
#     c.cd(i+1)
#     htemps[1][i].Draw()
# #     htemps[i].Fit('gaus', 'Q')

# c.Draw()

In [ ]:
ROOT.gStyle.SetOptStat()

# Multiplicity

In [ ]:
# dt1c = np.zeros((N, 2))

# dt1c[xyc_pad1_qa, 0] = 1e3*dt1[xyc_pad1_qa]-expo2_np(mm1_qall[xyc_pad1_qa], pars1in1_qa)
# dt1c[xyc_pad1_qa, 1] = expo2_np(mm1_qall[xyc_pad1_qa], spars1in1_qa)

In [ ]:
class share_fun():
    def __init__(self, times, resols):
        self.times = times
        self.resols = resols
    def __call__(self, params):
        test = params[0]
        res = np.sum( np.square( test - self.times ) / np.square( self.resols ) )
        return res

